<p style="text-align:center; font-size:60px; font-weight:bold;">NorthWind</p>


<h2 style="font-size:40px; font-weight:bold;">1 : Python Librairies</h2>

In [1]:
# This section imports the libraries we will use for SQLite queries in Python : 

import sqlite3
import pandas as pd
from IPython.display import display

---------------------------------------------------------------------------------------------------------------------------------------------------------

<h2 style="font-size:40px; font-weight:bold;">2 : Opening the tables from NorthWind</h2>

In [2]:
# Connection
conn = sqlite3.connect('northwind.db')

In [3]:
# Create a cursor to run SQL commands
cursor = conn.cursor()

In [4]:
# Liste des tables :
tables = "SELECT name FROM sqlite_master WHERE type='table';"
df_tables = pd.read_sql_query(tables, conn)
list_tables = list(df_tables["name"])
df_tables.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
name,Categories,sqlite_sequence,CustomerCustomerDemo,CustomerDemographics,Customers,Employees,EmployeeTerritories,Order Details,Orders,Products,Regions,Shippers,Suppliers,Territories


---------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
# définition de lecture de requêtes SQL en python :

def requests_sql(requete, conn):
    """
    Exécute une requête SQL et retourne un DataFrame pandas.
    """
    return pd.read_sql(requete, conn)

In [6]:
# Visualisation simplifiée de toutes les tables :

for name_table in list_tables :
    SQL = pd.read_sql(f"SELECT * FROM '{name_table}'", conn).head(1)
    display(name_table, SQL)

'Categories'

,CategoryID,CategoryName,Description,Picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...


'sqlite_sequence'

,name,seq
0,Categories,8


'CustomerCustomerDemo'

...

,ShipperID,CompanyName,Phone
0,1,Speedy Express,(503) 555-9831


'Suppliers'

,SupplierID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,British Isles,EC1 4SD,UK,(171) 555-2222,None,None


'Territories'

,TerritoryID,TerritoryDescription,RegionID
0,01581,Westboro,1


<h2 style="font-size:40px; font-weight:bold;">3 : SQL EXERCICES</h2>

## Exercices du 07 Juillet : 

In [7]:
# Exercice N°1 - 
# Top 5 clients par montant total des ventes avec nom du commercial et pays : 

query = ''' 
SELECT 
    Customers.CompanyName, 
    Customers.Country, 
    Employees.LastName || ' ' || Employees.FirstName AS EmployeeFullName, 
    ROUND(SUM(("Order Details".UnitPrice - "Order Details".Discount) * "Order Details".Quantity), 2) AS TotalSpend
FROM Customers
JOIN Orders ON Orders.CustomerID = Customers.CustomerID
JOIN Employees ON Employees.EmployeeID = Orders.EmployeeID
JOIN "Order Details" ON "Order Details".OrderID = Orders.OrderID
GROUP BY Customers.CompanyName, Customers.Country, EmployeeFullName
ORDER BY TotalSpend DESC
LIMIT 5
'''
df = requests_sql(query, conn)
df.head()

,CompanyName,Country,EmployeeFullName,TotalSpend
0,IT,None,Suyama Michael,1275827.66
1,IT,None,Callahan Laura,1267955.34
2,IT,None,Dodsworth Anne,1197330.37
3,IT,None,King Robert,1152356.43
4,Ana Trujillo Emparedados y helados,Mexico,Callahan Laura,1094612.21


In [8]:
# Exercice N°2 - 
# Pour chaque employé : total des ventes, nombre de commandes, classement par ventes (y compris ceux sans commande) : 

query = ''' 
SELECT
    employees.LastName || ' ' || employees.FirstName AS NomCompletEmployee,
    COALESCE(SUM( (COALESCE("order details".UnitPrice,0) - COALESCE("order details".Discount,0)) * COALESCE("order details".Quantity,0)), 0) AS TotalVente,
    COUNT(DISTINCT orders.OrderID) AS NombreVente,
    RANK() OVER (ORDER BY COALESCE(SUM( (COALESCE("order details".UnitPrice,0) - COALESCE("order details".Discount,0)) * COALESCE("order details".Quantity,0)), 0) DESC) AS Ranking
FROM employees
LEFT JOIN orders ON orders.EmployeeID = employees.EmployeeID
LEFT JOIN "order details" ON "order details".OrderID = orders.OrderID
GROUP BY NomCompletEmployee
ORDER BY TotalVente DESC 
'''

df = requests_sql(query, conn)
df.head()

,NomCompletEmployee,TotalVente,NombreVente,Ranking
0,Peacock Margaret,51504985.15,1908,1
1,Buchanan Steven,51393011.52,1804,2
2,Leverling Janet,50455397.27,1846,3
3,Davolio Nancy,49668983.33,1846,4
4,King Robert,49668215.56,1789,5


In [9]:
# Exercice N°3 - 
# Lister chaque catégorie de produit avec le total des ventes et le pourcentage des ventes globales : 

query = ''' 
WITH venteglobale AS (
    SELECT 
        SUM(("Order Details".UnitPrice - Discount) * Quantity) AS TotaleVentes
    FROM 'Order Details'
)
SELECT 
    categoryname,
    SUM(("Order Details".UnitPrice - Discount) * Quantity) AS TotaleVentesCategories,
    ROUND((SUM(('Order Details'.UnitPrice - Discount) * Quantity) / venteglobale.TotaleVentes) * 100, 2) AS PourcentageVentesCategories
FROM categories
JOIN products ON products.categoryid = categories.categoryid
JOIN "Order Details" ON "Order Details".productid = products.productid
, venteglobale
GROUP BY categoryname
'''

df = requests_sql(query, conn)
df.head()

,CategoryName,TotaleVentesCategories,PourcentageVentesCategories
0,Beverages,92181173.25,20.55
1,Condiments,55802440.13,12.44
2,Confections,66347063.04,14.79
3,Dairy Products,58034342.23,12.94
4,Grains/Cereals,28573291.59,6.37


In [10]:
# Exercice N°4 - 
# Afficher tous les clients avec la date de leur commande la plus récente (si elle existe) : 

query = ''' 
SELECT
    CompanyName,
    MAX(OrderDate) AS DateCommande
FROM customers
LEFT JOIN orders ON customers.customerID = orders.customerID
GROUP BY CompanyName
'''

df = requests_sql(query, conn)
df.head()

,CompanyName,DateCommande
0,Alfreds Futterkiste,2023-10-15 09:15:19
1,Ana Trujillo Emparedados y helados,2023-10-23 11:35:41
2,Antonio Moreno Taquería,2023-10-22 08:20:18
3,Around the Horn,2023-09-04 03:43:14
4,B's Beverages,2023-10-18 17:27:12


In [11]:
# Exercice N°5 - 
# Lister les commandes avec le nom du client, la date de commande, le montant total et la classification en 'Haute' ou 'Faible' valeur (selon un seuil) : 

query = ''' 
WITH TotalVenteCompany AS
(SELECT
    CompanyName,
    OrderDate,
    SUM((unitprice-discount) * quantity) AS TotalVente
FROM customers
JOIN orders ON orders.customerid = customers.customerid
JOIN "order details" ON "order details".orderid = orders.orderid
GROUP BY orders.orderid),

MeanVenteCompany AS
(SELECT
    CompanyName,
    OrderDate,
    AVG((unitprice-discount) * quantity) AS MeanVente
FROM customers
JOIN orders ON orders.customerid = customers.customerid
JOIN "order details" ON "order details".orderid = orders.orderid
GROUP BY CompanyName)

SELECT
    TotalVenteCompany.CompanyName,
    CASE
        WHEN TotalVente >= MeanVente THEN "Haute"
        ELSE "Faible"
        END
        AS BonneMauvaiseVente
FROM TotalVenteCompany
JOIN MeanVenteCompany On MeanVenteCompany.CompanyName = TotalVenteCompany.CompanyName
'''

df = requests_sql(query, conn)
df.head()

,CompanyName,BonneMauvaiseVente
0,Vins et alcools Chevalier,Faible
1,Toms Spezialitäten,Haute
2,Hanari Carnes,Haute
3,Victuailles en stock,Faible
4,Suprêmes délices,Haute


## Exercices du 08 Juillet : 

In [12]:
# Exercice N°1 - 
# Filtrer les commandes dont la date est entre deux dates données (BETWEEN) et dont le client est dans une liste précise (IN) : 

query = ''' 
SELECT
    orders.OrderID,
    DATE(orders.OrderDate) AS OrderDate,
    customers.CompanyName,
    customers.Country
FROM orders
JOIN customers ON customers.CustomerID = orders.CustomerID
WHERE DATE(orders.OrderDate) BETWEEN '2016-08-04' AND '2016-09-04'
  AND customers.CompanyName IN ('QUICK-Stop', 'Vins et alcools Chevalier')
ORDER BY orders.OrderDate
'''

df = requests_sql(query, conn)
df.head()

,OrderID,OrderDate,CompanyName,Country
0,10273,2016-08-05,QUICK-Stop,Germany
1,10274,2016-08-06,Vins et alcools Chevalier,France
2,10285,2016-08-20,QUICK-Stop,Germany
3,10286,2016-08-21,QUICK-Stop,Germany
4,24970,2016-08-24,QUICK-Stop,Germany


In [13]:
# Exercice N°2 - 
# Trouver les produits dont le nom contient un certain mot-clé (LIKE avec wildcard) et qui coûtent plus de 50 unités : 

query = ''' 
SELECT
    productname, 
    UnitPrice
FROM products
WHERE 
    (productname LIKE 'C%' 
     OR productname LIKE 'J%' 
     OR productname LIKE 'K%' 
     OR productname LIKE '%e')
  AND UnitPrice >= 50

'''

df = requests_sql(query, conn)
df.head()

,ProductName,UnitPrice
0,Carnarvon Tigers,62.5
1,Sir Rodney's Marmalade,81.0
2,Côte de Blaye,263.5


In [14]:
# Exercice N°3 - 
# Pour chaque client, afficher son total de commandes et dire ‘Actif’ ou ‘Inactif’ selon s’il dépasse la moyenne des commandes de tous les clients : 

query = ''' 
SELECT
    customers.CompanyName,
    SUM( ("Order Details".UnitPrice - "Order Details".Discount) * "Order Details".Quantity ) AS TotalVenteClient,
    CASE
        WHEN SUM( ("Order Details".UnitPrice - "Order Details".Discount) * "Order Details".Quantity )
            >= (
                SELECT AVG(ClientTotal) FROM (
                    SELECT
                        orders.CustomerID,
                        SUM( ("Order Details".UnitPrice - "Order Details".Discount) * "Order Details".Quantity ) AS ClientTotal
                    FROM orders
                    JOIN "Order Details" ON "Order Details".OrderID = orders.OrderID
                    GROUP BY orders.CustomerID
                )
            )
        THEN 'Actif'
        ELSE 'Inactif'
    END AS BonClient
FROM customers
JOIN orders ON orders.CustomerID = customers.CustomerID
JOIN "Order Details" ON "Order Details".OrderID = orders.OrderID
GROUP BY customers.CompanyName
ORDER BY TotalVenteClient DESC
'''

df = requests_sql(query, conn)
df.head()

,CompanyName,TotalVenteClient,BonClient
0,IT,9745371.29,Actif
1,B's Beverages,6154115.34,Actif
2,Hungry Coyote Import Store,5698023.67,Actif
3,Rancho grande,5559110.08,Actif
4,Gourmet Lanchonetes,5552584.15,Actif


In [15]:
# Exercice N°4 - 
# Lister les commandes dont le montant total est supérieur à la moyenne de toutes les commandes (Subquery dans WHERE avec IN / NOT IN ou >=) :

query = ''' 
SELECT
    *,
    SUM( ("Order Details".UnitPrice - "Order Details".Discount) * "Order Details".Quantity ) AS MontantCommande
FROM "order details"
GROUP BY orderid
HAVING MontantCommande >= (SELECT
        AVG( ("Order Details".UnitPrice - "Order Details".Discount) * "Order Details".Quantity ) AS MoyenneVente
        FROM "order details")
ORDER BY MontantCommande DESC
'''

df = requests_sql(query, conn)
df.head()

,OrderID,ProductID,UnitPrice,Quantity,Discount,MontantCommande
0,13460,1,18.0,38,0.0,73431.42
1,22571,1,18.0,23,0.0,69796.71
2,21669,1,18.0,19,0.0,69502.71
3,14425,1,18.0,37,0.0,69353.99
4,12861,1,18.0,22,0.0,69289.37


In [16]:
# Exercice N°5 - 
# Calculer le total cumulé des ventes par date de commande (SUM() OVER() partitionné/ordonné) :

query = ''' 
WITH VentesJour AS (
    SELECT
        DATE(orderdate) AS Date,
        SUM( ("Order Details".UnitPrice - "Order Details".Discount) * "Order Details".Quantity ) AS MontantJour
    FROM orders
    JOIN "order details" ON "order details".orderid = orders.orderid
    GROUP BY DATE(orderdate)
    ORDER BY DATE(orderdate))

SELECT 
    *,
    SUM(MontantJour) OVER (ORDER BY Date ) AS MontantCumule

FROM VentesJour
'''

df = requests_sql(query, conn)
df.head()

,Date,MontantJour,MontantCumule
0,2012-07-10,53582.20,53582.20
1,2012-07-11,117397.11,170979.31
2,2012-07-12,166916.21,337895.52
3,2012-07-13,100934.89,438830.41
4,2012-07-14,73440.42,512270.83


## Exercices du 14 Juillet : 

In [17]:
# Exercice N°1 - 
# Trouver les clients sans commandes (IS NULL / LEFT JOIN) :

query = ''' 
SELECT 
    customers.CustomerID, 
    customers.CompanyName, 
    customers.Country
FROM customers
LEFT JOIN orders ON customers.CustomerID = orders.CustomerID
WHERE orders.OrderID IS NULL
ORDER BY customers.CompanyName
'''

df = requests_sql(query, conn)
df.head()

,CustomerID,CompanyName,Country


In [18]:
# Exercice N°2 - 
# Produits avec ventes supérieures à la moyenne :

query = ''' 
WITH TotalVentesProduits AS (
    SELECT
        ProductID,
        SUM( (UnitPrice - Discount) * Quantity ) AS TotalVenteProduit
    FROM "Order Details"
    GROUP BY ProductID
),
MoyenneVente AS (
    SELECT AVG(TotalVenteProduit) AS Moyenne FROM TotalVentesProduits
)
SELECT
    p.ProductID,
    p.ProductName,
    t.TotalVenteProduit
FROM products p
JOIN TotalVentesProduits t ON p.ProductID = t.ProductID
JOIN MoyenneVente m
WHERE t.TotalVenteProduit > m.Moyenne
'''

df = requests_sql(query, conn)
df.head()

,ProductID,ProductName,TotalVenteProduit
0,7,Uncle Bob's Organic Dried Pears,6118659.45
1,8,Northwoods Cranberry Sauce,8074611.80
2,9,Mishi Kobe Niku,19424621.10
3,10,Ikura,6239952.70
4,12,Queso Manchego La Pastora,7684974.65


In [19]:
# Exercice N°3 - 
# Produit le plus cher par catégorie (correlated subquery) :

query = '''
WITH PrixMaxCat AS (
    SELECT
        products.Categoryid,
        MAX(products.Unitprice) AS MaxCategorie
    FROM products
    GROUP BY products.CategoryID)
        
SELECT
    products.productid,
    products.productname,
    products.unitprice,
    products.categoryid
FROM products
JOIN PrixMaxCat ON PrixMaxCat.categoryid = products.categoryid
WHERE products.unitprice = PrixMaxCat.maxcategorie
ORDER BY products.categoryid
'''

df = requests_sql(query, conn)
df.head()

,ProductID,ProductName,UnitPrice,CategoryID
0,38,Côte de Blaye,263.5,1
1,63,Vegie-spread,43.9,2
2,20,Sir Rodney's Marmalade,81.0,3
3,59,Raclette Courdavault,55.0,4
4,56,Gnocchi di nonna Alice,38.0,5


In [20]:
# Exercice N°4 - 
# Liste des employés avec concaténation de texte :

query = '''
SELECT 
    employeeid,
    lastname || ' ' || firstname AS NomCompletEmployees,
    title
FROM employees
ORDER BY lastname
'''

df = requests_sql(query, conn)
df.head()

,EmployeeID,NomCompletEmployees,Title
0,5,Buchanan Steven,Sales Manager
1,8,Callahan Laura,Inside Sales Coordinator
2,1,Davolio Nancy,Sales Representative
3,9,Dodsworth Anne,Sales Representative
4,2,Fuller Andrew,"Vice President, Sales"


In [21]:
# Exercice N°5 - 
# Rapport des ventes par pays avec commentaires SQL clairs :
# On calcule le total des ventes (UnitPrice - Discount) * Quantity.
# Jointure entre Orders et Order Details.
# Agrégation par ShipCountry.
# Tri décroissant sur le total des ventes.


query = '''
SELECT 
    orders.shipcountry,
    ROUND(SUM(( "order details".unitprice - "order details".discount) * "order details".quantity)) AS TotalVentePays
FROM orders
JOIN "order details" ON "order details".orderid = orders.orderid
GROUP BY orders.shipcountry
ORDER BY TotalVentePays DESC
'''

df = requests_sql(query, conn)
df.head()

,ShipCountry,TotalVentePays
0,USA,63873638.0
1,Germany,59108509.0
2,France,49125366.0
3,Brazil,46257888.0
4,UK,36259221.0


In [22]:
# Exercice N°5 - 
# : Analyse des ventes par employé et catégorie


query = '''
SELECT 
    employees.employeeid,
    employees.lastname || ' ' || employees.firstname AS NomCompleEmployee,
    categories.categoryid,
    categories.categoryname,
    SUM(("order details".UnitPrice - "order details".Discount) * "order details".Quantity) VentetotalEmployee    
FROM employees
JOIN orders On orders.employeeid = employees.employeeid
JOIN "order details" ON "order details".orderid = orders.orderid
JOIN products ON products.productid = "order details".productid
JOIN categories ON categories.categoryid = products.categoryid
GROUP BY products.categoryid, NomCompleEmployee
HAVING SUM("order details".Quantity) >= 1000
'''

df = requests_sql(query, conn)
df.head()

,EmployeeID,NomCompleEmployee,CategoryID,CategoryName,VentetotalEmployee
0,5,Buchanan Steven,1,Beverages,10615001.30
1,8,Callahan Laura,1,Beverages,9970718.60
2,1,Davolio Nancy,1,Beverages,10222410.05
3,9,Dodsworth Anne,1,Beverages,10272935.90
4,2,Fuller Andrew,1,Beverages,9538018.50


## Exercices du 17 Juillet : 

In [23]:
# Exercice N°1 - 
# Liste tous les produits qui n'ont jamais été vendus (c'est-à-dire qui n’apparaissent dans aucune commande) :

query = '''
SELECT 
    products.productid,
    products.productname,
    categories.categoryname,
    COUNT("order details".orderid) AS NbCommande
FROM products
JOIN categories ON products.categoryid = categories.categoryid
LEFT JOIN "order details" ON products.productid = "order details".productid
GROUP BY products.productid, categories.categoryname
HAVING NbCommande = 0
'''

df = requests_sql(query, conn)
df.head()

,ProductID,ProductName,CategoryName,NbCommande


In [24]:
# Exercice N°2 - 
# Classement des employés selon la diversité des produits vendus :

query = '''
SELECT
    employees.lastname || ' ' || employees.firstname AS NomCompletEmployee,
    COUNT(DISTINCT  "order details".productid) AS NbPorduitsDifferents,
    RANK() OVER(ORDER BY COUNT(DISTINCT  "order details".productid)) AS Rang
FROM employees
JOIN orders ON employees.employeeid = orders.employeeid
JOIN "order details" ON orders.orderid = "order details".orderid
GROUP BY NomCompletEmployee
'''

df = requests_sql(query, conn)
df.head()

,NomCompletEmployee,NbPorduitsDifferents,Rang
0,Buchanan Steven,77,1
1,Callahan Laura,77,1
2,Davolio Nancy,77,1
3,Dodsworth Anne,77,1
4,Fuller Andrew,77,1


In [25]:
# Exercice N°3 - 
# Meilleure catégorie de produit par pays  :

query = '''
WITH rankingpays AS (
    SELECT
        orders.shipcountry,
        categories.categoryname,
        SUM(("order details".unitprice - "order details".discount) * "order details".quantity) AS TotalVente,
        RANK() OVER(PARTITION BY orders.shipcountry ORDER BY SUM(("order details".unitprice - "order details".discount) * "order details".quantity) DESC) AS Rang
    FROM orders
    JOIN "order details" ON orders.orderid = "order details".orderid
    JOIN products ON "order details".productid = products.productid
    JOIN categories ON products.categoryid = categories.categoryid
    GROUP BY orders.shipcountry, categories.categoryname)

SELECT 
    *
FROM rankingpays
WHERE Rang = 1
'''

df = requests_sql(query, conn)
df.head()

,shipcountry,categoryname,TotalVente,Rang
0,Argentina,Beverages,3092820.50,1
1,Austria,Beverages,2118987.00,1
2,Belgium,Beverages,1912915.60,1
3,Brazil,Beverages,9571052.95,1
4,Canada,Beverages,3059096.90,1


In [26]:
# Exercice N°4 - 
# Catégories actives vs. inactives :

query = '''
SELECT 
    categories.categoryid,
    categories.categoryname,
    CASE 
        WHEN COUNT("order details".orderid) > 0 THEN 'Actif'
        ELSE 'Inactif'
    END AS StatutCategorie
FROM categories
JOIN products ON categories.categoryid = products.categoryid 
LEFT JOIN "order details" ON products.productid = "order details".productid
GROUP BY categories.categoryid, categories.categoryname
ORDER BY StatutCategorie DESC
'''

df = requests_sql(query, conn)
df.head()

,CategoryID,CategoryName,StatutCategorie
0,1,Beverages,Actif
1,2,Condiments,Actif
2,3,Confections,Actif
3,4,Dairy Products,Actif
4,5,Grains/Cereals,Actif


In [27]:
# Exercice N°5 - 
# Lister les clients ayant passé plus de 3 commandes, mais jamais en 1997 :

query = '''
SELECT 
    customers.companyname,
    COUNT(orders.orderid) AS nbCommande
FROM customers
JOIN orders ON customers.customerid = orders.customerid
WHERE customers.customerid NOT IN (
    SELECT DISTINCT customerid
    FROM orders
    WHERE SUBSTR(orderdate, 1, 4) = '1997'
)
GROUP BY customers.companyname
HAVING COUNT(orders.orderid) > 3
ORDER BY nbCommande DESC
'''

df = requests_sql(query, conn)
df.head()

,CompanyName,nbCommande
0,IT,335
1,B's Beverages,210
2,Ricardo Adocicados,203
3,LILA-Supermercado,203
4,Gourmet Lanchonetes,202


## Exercices du 22 Juillet : 

In [28]:
# Exercice N°1 - 
# Liste tous les produits :
    # dont le nom contient des caractères non alphabétiques (ex : chiffres, %, /, etc.)
    # ou dont le nom commence par un espace ou un tiret
    # ou dont le nom fait moins de 4 caractères :

query = '''
SELECT 
    productid,
    productname
FROM products
WHERE 
    productname GLOB '*[0-9]*'
    OR productname LIKE ' %'
    OR productname LIKE '-%'
    OR LENGTH(productname) < 4
'''

df = requests_sql(query, conn)
df.head()

,ProductID,ProductName


In [29]:
# Exercice N°2 - 
# Liste tous les clients :
    # dont le nom de société (CompanyName) contient un @ ou un .com
    # ou dont le nom de contact (ContactName) contient des chiffres
    # ou dont l’adresse (Address) fait moins de 10 caractères
    # trie le tout par pays et nom de société

query = '''
SELECT 
    customerid,
    companyname,
    contactname,
    country,
    address
FROM customers
WHERE 
    companyname LIKE '%@%' 
    OR companyname LIKE '%.com%' 
    OR contactname GLOB '*[0-9]*' 
    OR LENGTH(address) < 10
ORDER BY country, companyname
'''

df = requests_sql(query, conn)
df.head()

,CustomerID,CompanyName,ContactName,Country,Address
0,Val2,IT,Val2,None,None


In [30]:
# Exercice N°3 - 
# Trouver les employés avec données incohérentes ou incomplètes
# Liste tous les employés qui remplissent au moins une des conditions suivantes :
    # leur nom ou prénom est vide (NULL) ou de longueur < 2
    # leur titre (Title) contient le mot “Assistant” mais pas le mot “Manager”
    # leur Region est vide ou contient uniquement des espaces
    # leur BirthDate est après l’an 2000 (anomalie)

query = '''
SELECT
    *
FROM employees
WHERE 
    lastname IS NULL 
    OR firstname IS NULL
    OR LENGTH(lastname) < 2
    OR LENGTH(firstname) < 2
    OR (title LIKE '%Assistant%' AND title NOT LIKE '%Manager%')
    OR region IS NULL 
    OR TRIM(region) = ''
    OR SUBSTR(birthdate, 1, 4) > '2000'
'''

df = requests_sql(query, conn)
df.head()

,EmployeeID,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,ReportsTo,PhotoPath


In [31]:
# Exercice N°4 - 
# Détection de fournisseurs douteux ou incomplets : 
    # Liste tous les fournisseurs (suppliers) :
        # dont le CompanyName contient des caractères spéciaux (&, /, @, etc.)
        # ou dont le Phone contient moins de 8 chiffres
        # ou dont le Country est NULL ou égal à 'Unknown' (en insensible à la casse)
    # Trie les résultats par nom de fournisseur.

query = '''
SELECT 
    *
FROM suppliers
WHERE 
    companyname GLOB '*[&/@.-,]*'
    OR LENGTH(phone) < 8
    OR country IS NULL
    OR LOWER(country) = 'unknown'
ORDER BY companyname
'''

df = requests_sql(query, conn)
df.head()

,SupplierID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,14,Formaggi Fortini s.r.l.,Elio Rossi,Sales Representative,"Viale Dante, 75",Ravenna,Southern Europe,48100,Italy,(0544) 60323,(0544) 60603,#FORMAGGI.HTM#
1,11,Heli Süßwaren GmbH & Co. KG,Petra Winkler,Sales Manager,Tiergartenstraße 5,Berlin,Western Europe,10785,Germany,(010) 9984510,None,None
2,26,Pasta Buttini s.r.l.,Giovanni Giudici,Order Administrator,"Via dei Gelsomini, 153",Salerno,Southern Europe,84100,Italy,(089) 6547665,(089) 6547667,None
3,7,"Pavlova, Ltd.",Ian Devling,Marketing Manager,74 Rose St.\nMoonie Ponds,Melbourne,Victoria,3058,Australia,(03) 444-2343,(03) 444-6588,None
4,8,"Specialty Biscuits, Ltd.",Peter Wilson,Sales Representative,29 King's Way,Manchester,British Isles,M14 GSD,UK,(161) 555-4448,None,None


In [32]:
# Exercice N°5 - 
# Liste tous les produits :
    # dont le UnitPrice est zéro ou négatif
    # ou dont le UnitsInStock est NULL ou négatif
    # ou dont le UnitsOnOrder est supérieur au stock actuel
# Trie les résultats par UnitPrice croissant puis par ProductName

query = '''
SELECT 
    productid,
    productname,
    unitprice,
    unitsinstock,
    unitsonorder
FROM
    products
WHERE
    unitprice <= 0
    OR unitsinstock IS NULL 
    OR unitsinstock < 0
    OR unitsonorder > unitsinstock
ORDER BY unitprice, productname
'''

df = requests_sql(query, conn)
df.head()

,ProductID,ProductName,UnitPrice,UnitsInStock,UnitsOnOrder
0,45,Rogede sild,9.5,5,70
1,3,Aniseed Syrup,10.0,13,70
2,74,Longlife Tofu,10.0,4,20
3,21,Sir Rodney's Scones,10.0,3,40
4,31,Gorgonzola Telino,12.5,0,70


In [33]:
# Exercice N°6 - 
# Évolution des stocks : anomalies & suivi
    # Pour chaque produit :
        # Affiche son ProductName, UnitsInStock, UnitsOnOrder et ReorderLevel
    # Calcule un statut :
        # "À commander" si le stock est < au seuil (ReorderLevel) ET qu’aucune commande n’est en cours (UnitsOnOrder = 0)
        # "Déjà en commande" si le stock est < au seuil MAIS il y a des unités commandées
        # "Stock suffisant" dans les autres cas

query = '''
SELECT 
    productname,
    unitsinstock,
    unitsonorder,
    reorderlevel,
    CASE
        WHEN unitsinstock < reorderlevel AND unitsonorder = 0 THEN "A commander"
        WHEN unitsinstock < reorderlevel AND unitsonorder > 0 THEN "Déja en commande"
        ELSE "Stock suffisant"
        END AS Statut
FROM
    products  
'''

df = requests_sql(query, conn)
df.head()

,ProductName,UnitsInStock,UnitsOnOrder,ReorderLevel,Statut
0,Chai,39,0,10,Stock suffisant
1,Chang,17,40,25,Déja en commande
2,Aniseed Syrup,13,70,25,Déja en commande
3,Chef Anton's Cajun Seasoning,53,0,0,Stock suffisant
4,Chef Anton's Gumbo Mix,0,0,0,Stock suffisant


## Exercices du 28 Juillet : 

In [34]:
# Exercice N°1 - 
# Pour chaque pays dans lequel on a des clients, affiche :
    # Le nom du pays (Country)
    # Le nombre de clients dans ce pays
    # Le nombre total de commandes effectuées par ces clients
    # Le nombre de commandes non livrées (ShippedDate IS NULL)

query = '''
SELECT 
    c.Country,
    COUNT(DISTINCT c.CustomerID) AS NbClients,
    COUNT(o.OrderID) AS NbCommandes,
    SUM(CASE WHEN o.ShippedDate IS NULL THEN 1 ELSE 0 END) AS NbNonLivrees
FROM Customers c
LEFT JOIN Orders o ON c.CustomerID = o.CustomerID
GROUP BY c.Country
ORDER BY NbCommandes DESC, c.Country;
'''

df = requests_sql(query, conn)
df.head()

,Country,NbClients,NbCommandes,NbNonLivrees
0,USA,12,2104,3
1,France,12,2085,2
2,Germany,11,1895,2
3,Brazil,9,1659,2
4,UK,7,1234,0


In [35]:
# Exercice N°2 - 
 # Le manager veut identifier, pour chaque pays, le vendeur (employé) ayant généré le plus grand nombre de commandes.

query = '''
WITH NbCommandePays AS 
    (
    SELECT
        orders.shipcountry AS Country,
        lastname || ' ' || firstname AS NomCompletEmployee,
        COUNT(orders.orderid) AS NbCommande,
        RANK() OVER(PARTITION BY orders.shipcountry ORDER BY COUNT(orders.orderid) DESC) AS Ranking
    FROM 
        employees
    JOIN 
        orders ON orders.employeeid = employees.employeeid
    GROUP BY NomCompletEmployee, orders.shipcountry
    )

SELECT 
    country,
    nomcompletemployee,
    nbcommande
FROM
    NbCommandePays
WHERE 
    Ranking = 1
ORDER BY
    country
'''

df = requests_sql(query, conn)
df.head()

,Country,NomCompletEmployee,NbCommande
0,Argentina,Callahan Laura,75
1,Austria,Dodsworth Anne,48
2,Austria,King Robert,48
3,Belgium,Buchanan Steven,44
4,Brazil,Buchanan Steven,209


In [36]:
# Exercice N°3 - 
    # Le service logistique souhaite identifier les produits qui se vendent bien.
    # Il veut une liste des produits ayant été commandés plus de 100 fois, ainsi que :
        # Le nom du produit
        # Le nombre total de commandes dans lesquelles il apparaît
        # Le total de quantités commandées
        # Le prix unitaire (pour info)

query = '''
SELECT
    products.productname,
    COUNT(orders.orderid) AS NbCommande,
    SUM("order details".quantity) AS QuantiteTotale,
    products.unitprice AS PrixUnitaire
FROM
    orders
JOIN 
    "order details" ON orders.orderid = "order details".orderid
JOIN 
    products ON products.productid = "order details".productid
GROUP BY 
    products.productname, products.unitprice
HAVING 
    COUNT(DISTINCT orders.OrderID) > 100
ORDER BY 
    QuantiteTotale DESC;
'''

df = requests_sql(query, conn)
df.head()

,ProductName,NbCommande,QuantiteTotale,PrixUnitaire
0,Louisiana Hot Spiced Okra,8040,206213,17.00
1,Sir Rodney's Marmalade,7999,205637,81.00
2,Teatime Chocolate Biscuits,8024,205487,9.20
3,Sirop d'érable,7958,205005,28.50
4,Gumbär Gummibärchen,7999,204761,31.23


In [37]:
# Exercice N°4 - Produits & seuil de performance
# Pour chaque produit, afficher :
    # Le nom du produit
    # Le nombre total de commandes dans lesquelles il apparaît
    # La quantité totale commandée
    # Le prix unitaire
    # Une colonne "StatutPerformance" :
        # 'TOP' si le produit est dans les 5 plus commandés (en quantité totale)
        # 'HORS TOP' sinon

query = '''
WITH DetailsProduits AS 
    (
    SELECT
        products.productname AS ProductName,
        COUNT(orders.orderid) AS NbCommande,
        SUM("order details".quantity) AS QuantiteTotale,
        products.unitprice AS PrixUnitaire,
        RANK() OVER(ORDER BY SUM("order details".quantity) DESC) AS Ranking
    FROM
        orders
    JOIN 
        "order details" ON orders.orderid = "order details".orderid
    JOIN 
        products ON products.productid = "order details".productid
    GROUP BY 
        products.productname, products.unitprice
    ORDER BY 
        QuantiteTotale DESC
    )

SELECT 
    ProductName,
    NbCommande,
    QuantiteTotale,
    PrixUnitaire,
    CASE 
        WHEN ranking <= 5 THEN "Top"
        ELSE "Hors Top"
        END AS "StatutPerformance"
FROM 
    detailsproduits
'''

df = requests_sql(query, conn)
df.head()

,ProductName,NbCommande,QuantiteTotale,PrixUnitaire,StatutPerformance
0,Louisiana Hot Spiced Okra,8040,206213,17.00,Top
1,Sir Rodney's Marmalade,7999,205637,81.00,Top
2,Teatime Chocolate Biscuits,8024,205487,9.20,Top
3,Sirop d'érable,7958,205005,28.50,Top
4,Gumbär Gummibärchen,7999,204761,31.23,Top


In [38]:
# Exercice N°5 - Mise à jour des données client

# Un client important a déménagé à l'étranger.
# On te demande de :
    # 1. Trouver son CustomerID (nom de la société : 'Old World Delicatessen')
    # 2. Mettre à jour son pays ('Country') en le remplaçant par 'France'
    # 3. Vérifier que la mise à jour a bien été effectuée

df = pd.read_sql('''
    SELECT CustomerID, CompanyName, Country
    FROM Customers
    WHERE CompanyName = "Old World Delicatessen"
''', conn)

print(df.head())

cursor = conn.cursor()
cursor.execute('''
    UPDATE Customers
    SET Country = "France"
    WHERE CompanyName = "Old World Delicatessen"
''')
conn.commit()  # Obligatoire pour enregistrer les changements

df = pd.read_sql('''
    SELECT CustomerID, CompanyName, Country
    FROM Customers
    WHERE CompanyName = "Old World Delicatessen"
''', conn)

print(df.head())

  CustomerID             CompanyName Country
0      OLDWO  Old World Delicatessen  France
  CustomerID             CompanyName Country
0      OLDWO  Old World Delicatessen  France


In [39]:
# Exercice N°6 - Suppression sécurisée d'une entrée
# Un ancien client fictif a été ajouté par erreur dans la table Customers.
# Il faut le supprimer proprement de la base (version sandbox), en respectant les bonnes pratiques :
    # 1. Vérifier si le client 'Faux Client Test' existe dans la table Customers
    #    (sinon, l'insérer d'abord pour l'exercice).
    # 2. Supprimer ce client avec DELETE (en utilisant une clause WHERE précise).
    # 3. Vérifier que la suppression a bien été effectuée.

# Vérification de sa présence initiale :
df = pd.read_sql('''
    SELECT CustomerID, CompanyName, Country
    FROM Customers
    WHERE CompanyName = "Faux Client Test"
''', conn)
print(df.head())

# Ajout du faux client ou ignore si déjà présent :
cursor = conn.cursor()
cursor.execute('''
    INSERT OR IGNORE INTO Customers (CustomerID, CompanyName, Country)
    VALUES ('FAUX1', 'Faux Client Test', 'USA');
''')
conn.commit()

# Vérification de l'ajout :
df = pd.read_sql('''
    SELECT CustomerID, CompanyName, Country
    FROM Customers
    WHERE CompanyName = "Faux Client Test"
''', conn)
print(df.head())

# Suppression du faux clients :
cursor = conn.cursor()
cursor.execute('''
    DELETE FROM Customers
    WHERE CompanyName = 'Faux Client Test';
''')
conn.commit()

# Vérification de la suppression :
df = pd.read_sql('''
    SELECT CustomerID, CompanyName, Country
    FROM Customers
    WHERE CompanyName = "Faux Client Test"
''', conn)
print(df.head())

Empty DataFrame
Columns: [CustomerID, CompanyName, Country]
Index: []
  CustomerID       CompanyName Country
0      FAUX1  Faux Client Test     USA
Empty DataFrame
Columns: [CustomerID, CompanyName, Country]
Index: []


## Exercices du programme semaine 1 : 

In [40]:
# Exercice N°1 – Nombre de commandes par client
# --------------------------------------------------
# Contexte métier : L’équipe commerciale souhaite connaître les clients les plus actifs.
# Ce que tu dois retourner :
    # Le nom de la société (CompanyName)
    # Le nombre de commandes effectuées
# Tables concernées : Customers, Orders

query = '''
SELECT
    customers.companyname AS NomEntreprise,
    COUNT(orders.orderid) AS NbCommande
FROM customers
JOIN orders ON customers.customerid = orders.customerid
GROUP BY customers.companyname
ORDER BY NbCommande DESC
'''

df = requests_sql(query, conn)
df.head()

# Question entretien : Quelle clause permet de filtrer un résultat après une agrégation ?
# Réponse : HAVING sert à filtrer un résultat après une agrégation et WHERE si pas d'agrégation. 

,NomEntreprise,NbCommande
0,IT,335
1,B's Beverages,210
2,Ricardo Adocicados,203
3,LILA-Supermercado,203
4,Gourmet Lanchonetes,202


In [41]:
# Exercice N°2 – Quantité totale commandée par catégorie
# --------------------------------------------------
# Contexte métier : Le marketing souhaite connaître les catégories de produits les plus vendues.
# Ce que tu dois retourner :
    # Le nom de la catégorie
    # La quantité totale commandée de tous les produits de cette catégorie
# Tables concernées : Categories, Products, Order Details

query = '''
SELECT 
    categories.categoryname AS NomCategorie,
    SUM("order details".quantity) AS TotalQuantite
FROM
    categories
JOIN 
    products ON categories.categoryid = products.categoryid
JOIN 
    "order details" ON products.productid = "order details".productid
GROUP BY 
    categories.categoryname
ORDER BY 
    TotalQuantite DESC
'''

df = requests_sql(query, conn)
df.head()

# Question entretien : À quoi sert la clause GROUP BY ?
# Réponse : Elle sert à regrouper les données selon une clause (diminuer la granularité)

,NomCategorie,TotalQuantite
0,Confections,2628466
1,Beverages,2427361
2,Condiments,2420864
3,Seafood,2410782
4,Dairy Products,2020160


In [42]:
# Exercice N°3 – Classement des vendeurs par nombre de commandes
# --------------------------------------------------
# Contexte métier : Le manager veut voir les top vendeurs de chaque pays.
# Ce que tu dois retourner :
    # Le pays de l’employé
    # Le nom complet de l’employé
    # Le nombre de commandes
    # Son rang dans son pays
# Tables concernées : Employees, Orders

query = '''
SELECT 
    employees.lastname || ' ' || employees.firstname AS NomEmployee,
    employees.country,
    COUNT(orders.orderid) AS NbCommande,
    RANK() OVER(PARTITION BY country ORDER BY COUNT(orders.orderid) DESC) AS Ranking
FROM
    employees
JOIN 
    orders 
    ON employees.employeeid = orders.employeeid
GROUP BY 
    employees.lastname || ' ' || employees.firstname
ORDER BY 
    employees.country, ranking
'''

df = requests_sql(query, conn)
df.head()

# Question entretien : Quelle est la différence entre RANK() et ROW_NUMBER() ?
# Réponse : Le RANK est l'ordre qu'on lui donne selon une clause (mais ça peut changer si on partitionne ou pas), ROW_NUMBER est identique mais si il y a un double ils n'auront pas la même valeur. 


,NomEmployee,Country,NbCommande,Ranking
0,Buchanan Steven,UK,1804,1
1,King Robert,UK,1789,2
2,Dodsworth Anne,UK,1766,3
3,Suyama Michael,UK,1754,4
4,Peacock Margaret,USA,1908,1


In [43]:
# Exercice N°4 – Correction d'une requête cassée
# --------------------------------------------------
# Contexte métier : L’équipe veut afficher le nombre de commandes par pays, mais la requête ci-dessous est incorrecte.
    # Corrige-la.
# Tables concernées : Customers, Orders

query = '''
SELECT
    country, 
    COUNT(orderid) AS nbCommande
FROM 
    customers
JOIN 
    orders ON customers.customerid = orders.customerid
GROUP BY
    country
ORDER BY
    nbcommande DESC
'''

df = requests_sql(query, conn)
df.head()

# Question entretien : Quels sont les risques d’utiliser une jointure implicite ?
# Réponse : Qu'il ne trouve pas les clés primaires.

# Question entretien : Que fait GROUP BY ?
# Réponse : Ici il regroupe les données du nombre de commande par pays.

,Country,nbCommande
0,USA,2104
1,France,2085
2,Germany,1895
3,Brazil,1659
4,UK,1234


In [44]:
# Exercice N°5 – Statut promo des produits
# --------------------------------------------------
# Contexte métier : Le service achat veut savoir quels produits sont en promo.
# Ce que tu dois retourner :
    # Le nom du produit
    # Son prix unitaire
    # Un statut 'PROMO' si le prix est < 20, sinon 'STANDARD'
# Tables concernées : Products

query = '''
SELECT 
    products.productname AS NomProduit,
    products.unitprice AS PrixUnitaire,
    CASE
        WHEN products.unitprice < 20 THEN 'PROMO'
        ELSE 'STANDARD'
        END AS 'Statut'
FROM
    products
'''

df = requests_sql(query, conn)
df.head()

# Question entretien : Quelle clause utiliser pour créer une colonne conditionnelle ?
# Réponse : On utilise le CASE WHEN THEN ELSE END AS pour lui stipuler les différentes conditions, ce que l'on veut comparer et le nom ou valeur qu'on lui attribut ensuite.


,NomProduit,PrixUnitaire,Statut
0,Chai,18.00,PROMO
1,Chang,19.00,PROMO
2,Aniseed Syrup,10.00,PROMO
3,Chef Anton's Cajun Seasoning,22.00,STANDARD
4,Chef Anton's Gumbo Mix,21.35,STANDARD


In [45]:
# Exercice N°6 – Analyse de commandes par période
# --------------------------------------------------
# Contexte métier : Le responsable logistique veut analyser les commandes entre deux années.
# Ce que tu dois retourner :
    # L’année de la commande
    # Le nombre total de commandes cette année

# Tables concernées : Orders

query = '''
SELECT
    substring(orders.shippeddate, 0, 5) AS Année,
    COUNT(orders.orderid) AS NbCommande
FROM
    orders
GROUP BY
    substring(orders.shippeddate, 0, 5)
ORDER BY 
    nbcommande DESC
'''

df = requests_sql(query, conn)
df.head()

# Question entretien : Quelle fonction utiliser pour extraire l’année d’une date ?
# Réponse : On peut utiliser dans certains cas YEAR() mais ici en sqlite il faut utiliser une substring.

,Année,NbCommande
0,2017,1781
1,2018,1537
2,2016,1503
3,2015,1434
4,2021,1427


In [46]:
# Exercice N°7 – Meilleurs clients avec CTE
# --------------------------------------------------
# Contexte métier : On veut identifier les clients ayant passé plus de 5 commandes.
# Ce que tu dois retourner :
    # Le nom de la société
    # Le nombre de commandes
# Tables concernées : Customers, Orders

query = '''
WITH nbcommandecompanie AS 
    (
    SELECT
        customers.companyname AS NomCompanie,
        COUNT(orders.orderId) AS NbCommandes
    FROM
        customers
    JOIN
        orders
        ON customers.customerid = orders.customerid
    GROUP BY
        customers.companyname
    ORDER BY
        nbcommandes
    )
SELECT 
    *
FROM 
    nbcommandecompanie
WHERE
    nbcommandes > 5
'''

df = requests_sql(query, conn)
df.head()

# Question entretien : À quoi sert une CTE ? Quelle différence avec une sous-requête ?
# Réponse : A créer une ou plusieurs tables intermédiaires. Même si ici on pouvait le faire directement avec un HAVING.

,NomCompanie,NbCommandes
0,Océano Atlántico Ltda.,154
1,FISSA Fabrica Inter. Salchichas S.A.,155
2,Furia Bacalhau e Frutos do Mar,155
3,La corne d'abondance,155
4,Around the Horn,156


In [47]:
# Exercice N°8 – Segmentation des produits par prix
# --------------------------------------------------
# Contexte métier : Le marketing souhaite segmenter les produits selon leur prix.
# Ce que tu dois retourner :
    # Le nom du produit
    # Le prix unitaire
    # Une colonne "Segment" :
        # 'Bas' si < 20
        # 'Moyen' si entre 20 et 50
        # 'Haut' sinon
# Tables concernées : Products

query = '''
SELECT
    products.productname AS NomProduit,
    products.unitprice AS PrixUnitaire,
    CASE
        WHEN products.unitprice < 20 THEN 'BAS'
        WHEN products.unitprice > 50 THEN 'HAUT'
        ELSE 'MOYEN'
        END AS "Segment"
FROM
    products
'''

df = requests_sql(query, conn)
df.head()

# Question entretien : Quelle est la différence entre BETWEEN et >= / <= ?
# Réponse : Between c'est entre X et Y, et sinon on a suppérieur ou égale et inférieur ou égale à Z. Ici on en avait pas besoin.

,NomProduit,PrixUnitaire,Segment
0,Chai,18.00,BAS
1,Chang,19.00,BAS
2,Aniseed Syrup,10.00,BAS
3,Chef Anton's Cajun Seasoning,22.00,MOYEN
4,Chef Anton's Gumbo Mix,21.35,MOYEN


In [48]:
# Exercice N°9 – Correction d’une requête avec mauvaise agrégation
# --------------------------------------------------
# Contexte métier : L'équipe a écrit une requête pour compter les produits par catégorie, mais elle ne fonctionne pas.
# Corrige-la.

query = '''
SELECT 
    categories.categoryname AS NomCategorie, 
    COUNT(categories.categoryid) AS NbProduits, 
    ROUND(AVG(products.unitprice),1) AS MoyennePrixUnitaire
FROM 
    categories
JOIN 
    products 
    ON categories.categoryid = products.categoryid
GROUP BY 
    categories.categoryname
'''

df = requests_sql(query, conn)
df.head()

# Question entretien : Pourquoi certaines colonnes doivent-elles être dans le GROUP BY ?
# Réponse : C'est la colonne par laquelle les données sont agrégées. 

,NomCategorie,NbProduits,MoyennePrixUnitaire
0,Beverages,12,38.0
1,Condiments,12,23.1
2,Confections,13,25.2
3,Dairy Products,10,28.7
4,Grains/Cereals,7,20.3


In [49]:
# Exercice N°10 – Cas complet : commandes, clients et produits
# --------------------------------------------------
# Contexte métier : On souhaite analyser les 10 produits les plus commandés (en quantité totale), en listant :
    # Le nom du produit
    # La quantité totale vendue
    # Le nombre de clients différents ayant commandé ce produit
# Tables concernées : Products, Order Details, Orders, Customers

query = '''
SELECT
    products.productname AS NomProduit,
    SUM("order details".quantity) AS QuantiteTotal,
    COUNT(DISTINCT customers.customerid) AS NbClientUnique
FROM
    products
JOIN
    "order details"
    ON products.productid = "order details".productid
JOIN
    orders
    ON "order details".orderid = orders.orderid
JOIN
    customers
    ON orders.customerid = customers.customerid
GROUP BY
    products.productname
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Que ferait cette requête si on enlevait le COUNT DISTINCT ?
# Réponse : Elle compterait tous les clients même si un client à acheté plusieurs fois le produit.

# Questions entretien : Comment garantir une requête claire et maintenable ?
# Réponse : Avoir des bons alias, avoir une structure claire, ajouter des commentaires (ici c'est compliqué)

,NomProduit,QuantiteTotal,NbClientUnique
0,Alice Mutton,202304,93
1,Aniseed Syrup,202186,93
2,Boston Crab Meat,200232,93
3,Camembert Pierrot,203154,93
4,Carnarvon Tigers,201747,93


## Exercices du programme semaine 2 : 04 Aout 2025

In [50]:
# Exercice 1 – Produits les plus chers par catégorie
# --------------------------------------------------
# Produits les plus chers par catégorie :

query = '''
WITH ranking AS 
    (
    SELECT
        categories.categoryid,
        categories.categoryname,
        products.productid,
        products.productname,
        products.unitprice,
        RANK() OVER(PARTITION BY categories.categoryid ORDER BY products.unitprice DESC) AS Rang
    FROM
        categories
    JOIN
        products
        ON categories.categoryid = products.categoryid
    )
SELECT
    ranking.categoryname,
    ranking.productname,
    ranking.unitprice
FROM
    ranking
WHERE
    rang = 1
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle différence entre MAX() dans une SELECT classique et dans une PARTITION BY ?
# Réponse : Le MAX ans un select classic ou un RANK sans PARTITION va donner le max de la table pour cette colonne, mais dans un PARTITION BY il va donner le MAX pour chacun des termes de la colonne partitionnée.

# Questions entretien : Comment retourner uniquement les lignes correspondant à la valeur maximale d’un groupe ?
# Réponse : Avec un PARTITION BY qui va recommencer à 1 pour chaque groupe (ici les categories).

,categoryname,productname,unitprice
0,Beverages,Côte de Blaye,263.5
1,Condiments,Vegie-spread,43.9
2,Confections,Sir Rodney's Marmalade,81.0
3,Dairy Products,Raclette Courdavault,55.0
4,Grains/Cereals,Gnocchi di nonna Alice,38.0


In [51]:
# Exercice 2 – Clients n’ayant jamais passé commande
# --------------------------------------------------
# Produits les plus chers par catégorie :

query = '''
SELECT
    customers.companyname,
    COUNT(orders.orderid) AS nbcommandes
FROM
    customers
LEFT JOIN
    orders
    ON customers.customerid = orders.customerid
GROUP BY
    customers.customerid
HAVING
    nbcommandes = 0
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle différence entre NOT IN et LEFT JOIN IS NULL ?
# Réponse : NOT IN = Vérifie qu’une valeur n’apparaît pas dans une liste ou un résultat de sous-requête, LEFT JOIN IS NULL = Recherche l’absence de correspondance dans une table liée (typiquement pour détecter les “sans relation”).

# Questions entretien : Que se passe-t-il si la sous-requête retourne un NULL dans un NOT IN ?
# Réponse : Aucune ligne ne passe le filtre 

,CompanyName,nbcommandes


In [52]:
# Exercice 3 – Produits commandés par plus de 5 clients
# --------------------------------------------------
# Contexte métier : Le service commercial souhaite connaître les produits qui ont été commandés par une base client large.
# Ce que tu dois retourner :
    # Le nom du produit
    # Le nombre de clients différents ayant commandé ce produit
# Filtrer ensuite pour ne garder que ceux commandés par plus de 5 clients.
# Tables concernées : Products, Order Details, Orders, Customers

query = '''
SELECT
    products.productname,
    COUNT(DISTINCT customers.customerid) AS nbClients
FROM
    products
JOIN
    "order details" 
    ON products.productid = "order details".productid
JOIN
    orders
    ON "order details".orderid = orders.orderid
JOIN
    customers
    ON orders.customerid = customers.customerid
GROUP BY 
    products.productid
HAVING
    nbclients > 5
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : À quoi sert HAVING et pourquoi ne pas utiliser WHERE ici ?
# Réponse : On utilise HAVING pour filtrer un GROUP BY et un WHERE pour filtrer hors GROUP BY.

# Questions entretien : Comment dédupliquer des clients dans un COUNT() ?
# Réponse : En utilisant le DISTINCT qui va compter que les clients distincts. 

,ProductName,nbClients
0,Chai,93
1,Chang,93
2,Aniseed Syrup,93
3,Chef Anton's Cajun Seasoning,93
4,Chef Anton's Gumbo Mix,93


In [53]:
# Exercice 4 – [Requête cassée] – Sous-requête mal imbriquée
# --------------------------------------------------
# Contexte métier : L’équipe veut identifier les clients ayant passé plus de commandes que la moyenne.
# Mais la requête ci-dessous est incorrecte.
# Corrige-la.

query = '''
SELECT 
    customers.companyname,
    COUNT(orders.orderid) AS nbCommandes
FROM 
    customers
JOIN 
    orders ON customers.customerid = orders.customerid
GROUP BY 
    customers.customerid
HAVING 
    COUNT(orders.orderid) > (
        SELECT AVG(nb)
        FROM (
            SELECT COUNT(*) AS nb
            FROM orders
            GROUP BY customerid
        )
    )

'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Que fait une sous-requête dans WHERE ?
# Réponse : Elle permet de comparer chaque ligne du résultat à un calcul effectué séparément, souvent global.

# Questions entretien : Quelles erreurs classiques peut-on rencontrer avec COUNT() dans une clause WHERE ?
# Réponse : COUNT() est une agrégation, donc ne peut pas être utilisée dans WHERE directement — elle doit venir après un GROUP BY, ou être filtrée avec HAVING.

,CompanyName,nbCommandes
0,Ana Trujillo Emparedados y helados,195
1,Berglunds snabbköp,178
2,Bólido Comidas preparadas,182
3,Bottom-Dollar Markets,185
4,B's Beverages,210


In [54]:
# Exercice 5 – Date de première et dernière commande d’un client
# --------------------------------------------------
# Contexte métier : Le service relation client veut connaître l’ancienneté de ses clients.
# Ce que tu dois retourner :
    # Le nom de la société
    # La date de la première commande
    # La date de la dernière commande
# Tables concernées : Customers, Orders

query = '''
SELECT
    customers.companyname,
    MIN(DATE(orders.orderdate)) AS PremiereCommande,
    MAX(DATE(orders.orderdate)) AS DerniereCommande    
FROM
    customers
JOIN
    orders
    ON customers.customerid = orders.customerid
GROUP BY
    customers.companyname
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle fonction retourne la première ou dernière valeur ?
# Réponse : MAX() et MIN()

# Questions entretien : Pourquoi utiliser GROUP BY ici ?
# Réponse : Pour avoir les résultats pour chaque clients et non de la base de données entière.

,CompanyName,PremiereCommande,DerniereCommande
0,Alfreds Futterkiste,2012-08-22,2023-10-15
1,Ana Trujillo Emparedados y helados,2012-08-30,2023-10-23
2,Antonio Moreno Taquería,2012-07-13,2023-10-22
3,Around the Horn,2012-07-11,2023-09-04
4,B's Beverages,2012-08-16,2023-10-18


In [55]:
# Exercice 6 – Nombre de produits par commande
# --------------------------------------------------
# Contexte métier : Le service des opérations veut connaître le nombre de lignes produits par commande.
# Ce que tu dois retourner :
    # L’identifiant de la commande
    # Le nom de la société ayant passé la commande
    # Le nombre total de lignes de commande (produits différents)
# Tables concernées : Orders, Order Details, Customers

query = '''
SELECT
    orders.orderid,
    customers.companyname,
    COUNT(DISTINCT "order details".productid) AS NbProduits
FROM
    orders
JOIN
    "order details" 
    ON orders.orderid = "order details".orderid
JOIN
    customers
    ON orders.customerid = customers.customerid
GROUP BY
    orders.orderid
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Peut-on utiliser une sous-requête dans un champ SELECT ?
# Réponse : Oui, on peut utiliser une sous-requête dans un champ SELECT, notamment quand on veut calculer une valeur agrégée ou dépendante ligne par ligne.

# Questions entretien : Quelle est la différence entre une sous-requête corrélée et une sous-requête classique ?
# Réponse : Une sous-requête classique ne dépend pas de la ligne courante : elle peut être exécutée seule. Une sous-requête corrélée dépend de chaque ligne du résultat principal, donc elle est réévaluée à chaque itération.

,OrderID,CompanyName,NbProduits
0,10248,Vins et alcools Chevalier,3
1,10249,Toms Spezialitäten,2
2,10250,Hanari Carnes,3
3,10251,Victuailles en stock,3
4,10252,Suprêmes délices,3


In [56]:
# Exercice 7 – Produits dans une gamme de prix
# --------------------------------------------------
# Contexte métier : Le service achat souhaite identifier les produits situés dans une gamme de prix stratégique.
# Ce que tu dois retourner :
    # Le nom du produit
    # Le prix unitaire
# Filtrer pour ne garder que les produits dont le prix est entre 15 et 25 (inclus).
# Tables concernées : Products

query = '''
SELECT
    products.productname,
    products.unitprice
FROM
    products
WHERE
    products.unitprice BETWEEN 15 AND 25
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle est la différence entre BETWEEN et >= / <= ?
# Réponse : Plus lisible et compact.

# Questions entretien : Est-ce que BETWEEN inclut les bornes ?
# Réponse : OUI

,ProductName,UnitPrice
0,Chai,18.00
1,Chang,19.00
2,Chef Anton's Cajun Seasoning,22.00
3,Chef Anton's Gumbo Mix,21.35
4,Grandma's Boysenberry Spread,25.00


In [57]:
# Exercice 8 – Produits appartenant à certaines catégories
# --------------------------------------------------
# Contexte métier : Le service marketing souhaite se concentrer uniquement sur les produits des catégories ‘Beverages’, ‘Confections’ et ‘Seafood’.
# Ce que tu dois retourner :
    # Le nom du produit
    # Le prix unitaire
    # Le nom de la catégorie
# Tables concernées : Products, Categories

query = '''
SELECT
    products.productname,
    products.unitprice,
    categories.categoryname
FROM
    categories
JOIN 
    products
    ON categories.categoryid = products.categoryid
WHERE
    categories.categoryname IN ("Beverages", "Confections", "Seafood")
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle est la différence entre IN et plusieurs conditions OR ?
# Réponse : Il n'y en à pas c'est juste plus propre et plus lisible.

# Questions entretien : Que se passe-t-il si l’une des valeurs d’une sous-requête utilisée dans NOT IN est NULL ?
# Réponse : Si une sous-requête utilisée avec NOT IN retourne un ou plusieurs NULL, alors la condition entière échoue → aucune ligne ne sera retournée.

,ProductName,UnitPrice,CategoryName
0,Chai,18.00,Beverages
1,Chang,19.00,Beverages
2,Ikura,31.00,Seafood
3,Konbu,6.00,Seafood
4,Pavlova,17.45,Confections


In [58]:
# Exercice 9 – [Requête cassée] – Erreur d’agrégation sur les fournisseurs
# --------------------------------------------------
# Contexte métier : Le service achat veut connaître le nombre de produits vendus par chaque fournisseur.
# Mais la requête actuelle plante ou retourne de mauvais résultats.
# Corrige-la.

query = '''
SELECT 
    suppliers.companyname,
    COUNT(products.productid) AS nbProduits
FROM 
    suppliers
JOIN 
    products ON suppliers.supplierid = products.supplierid
GROUP BY 
    suppliers.companyname
ORDER BY 
    nbProduits DESC
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Pourquoi une erreur peut apparaître avec GROUP BY si on ne regroupe pas toutes les colonnes ?
# Réponse : En SQL strict (comme PostgreSQL), toute colonne dans le SELECT qui n’est pas agrégée doit apparaître dans le GROUP BY. Sinon, l’agrégation ne sait pas comment l’interpréter.

# Questions entretien : Quelle colonne doit apparaître dans SELECT quand on agrège ?
# Réponse : Les colonnes agrégées (COUNT, SUM, MAX, etc.), Les colonnes présentes dans le GROUP BY, Aucune colonne "intermédiaire" qui serait ambigüe (ex : un productname sans GROUP BY)

,CompanyName,nbProduits
0,Plutzer Lebensmittelgroßmärkte AG,5
1,"Pavlova, Ltd.",5
2,"Specialty Biscuits, Ltd.",4
3,New Orleans Cajun Delights,4
4,Tokyo Traders,3


In [59]:
# Exercice 10 – Clients actifs dans une période donnée
# --------------------------------------------------
# Contexte métier : Le service CRM souhaite identifier les clients ayant passé au moins une commande entre 2017 et 2018 (inclus).
# Ce que tu dois retourner :
    # Le nom de la société
    # Le pays
    # Le nombre total de commandes sur cette période
# Tables concernées : Customers, Orders

query = '''
SELECT
    customers.companyname,
    customers.country,
    COUNT(orders.orderid) AS nbcommande
FROM
    customers
JOIN
    orders
    ON customers.customerid = orders.customerid
WHERE
    SUBSTR(orders.orderdate, 0, 5) = "2017" OR  SUBSTR(orders.orderdate, 0, 5) = "2018"
GROUP BY
    customers.companyname
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle est la différence entre WHERE et HAVING dans un filtre sur date ?
# Réponse : WHERE filtre avant le GROUP BY et le HAVING filtre aprés le GROUP BY donc sur une moyenne ou une mesure ou un calcul.

# Questions entretien : Pourquoi utiliser COUNT(DISTINCT ...) dans certains cas ?
# Réponse : Avec DISTINCT il compte que une fois chaque client et sans il les compte plusieurs fois si ils ont fait plusieurs commandes.

,CompanyName,Country,nbcommande
0,Alfreds Futterkiste,Germany,31
1,Ana Trujillo Emparedados y helados,Mexico,36
2,Antonio Moreno Taquería,Mexico,24
3,Around the Horn,UK,32
4,B's Beverages,UK,47


## Exercices du programme semaine 2.5 : 11 Aout 2025

In [60]:
# Exercice 1 – Moyenne des commandes par client
# --------------------------------------------------
# Contexte métier :
# La direction veut savoir, pour chaque client, combien de commandes il a passées et comment ce chiffre se compare à la moyenne générale de commandes par client.
# Ce que tu dois retourner :
    # Le nom de la société (CompanyName)
    # Le nombre total de commandes passées par ce client
    # La moyenne générale de commandes par client (même valeur sur chaque ligne, via sous-requête en SELECT)
# Tables concernées : Customers, Orders

query = '''
SELECT 
    customers.companyname,
    COUNT(orders.orderid) AS nbcommandes,
    ROUND(
    (
        SELECT AVG(nb)
        FROM (
            SELECT COUNT(*) AS nb
            FROM Orders
            GROUP BY CustomerID
        )
    ),2) AS moyennenbcommandes
FROM
    customers
JOIN
    orders 
    ON customers.customerid = orders.customerid
GROUP BY
    customers.companyname
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle est la différence entre une sous-requête et une CTE ?
# Réponse : Une CTE (WITH) crée une table temporaire qui peut être appellée plusieurs fois (c'est beaucoup plus simple d'utilisation aussi), la sous requete elle va filtrer directement)

# Questions entretien : Peut-on utiliser une agrégation dans une sous-requête en SELECT ?
# Réponse : Oui, si la sous-requête renvoie une seule valeur (sous-requête scalaire).

,CompanyName,nbcommandes,moyennenbcommandes
0,Alfreds Futterkiste,163,175.08
1,Ana Trujillo Emparedados y helados,195,175.08
2,Antonio Moreno Taquería,163,175.08
3,Around the Horn,156,175.08
4,B's Beverages,210,175.08


In [61]:
# Exercice 2 – Top 5 catégories par ventes (sous-requête en FROM)
# --------------------------------------------------
# Contexte métier :
# Le marketing veut connaître les 5 catégories qui génèrent le plus de revenus.
# Ce que tu dois retourner :
    # Le nom de la catégorie
    # Le revenu total (SUM(Quantity * UnitPrice))
# Contraintes :
    # Calcule d’abord les revenus par catégorie dans une SOUS-REQUÊTE placée dans FROM
    # Puis, dans la requête externe, trie et limite aux 5 premières catégories
# Tables concernées : Categories, Products, "Order Details"

query = '''

SELECT 
    categoryname,
    revenuetotal
FROM
    (
    SELECT
        categories.categoryname,
        SUM("order details".quantity * "order details".unitprice * (1 - "order details".discount)) AS revenuetotal
    FROM
        categories
    JOIN
        products
        ON categories.categoryid = products.categoryid
    JOIN
        "order details"
        ON products.productid = "order details".productid
    GROUP BY
        categories.categoryname
    ) AS revenus_cat
ORDER BY
    revenuetotal DESC
LIMIT 
    5
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : À quoi sert une sous-requête dans FROM (a.k.a. table dérivée) ?
# Réponse : Elle permet de préparer un jeu de résultats intermédiaire (souvent avec un GROUP BY ou un calcul) que l’on va ensuite exploiter comme une table. 

# Questions entretien : Faut-il toujours donner un alias à une sous-requête dans FROM ? Pourquoi ?
# Réponse : Oui, il faut toujours donner un alias à une sous-requête dans FROM car SQL doit pouvoir l’identifier comme une "table" (même temporaire) et s’y référer.

,categoryname,revenuetotal
0,Beverages,9.216318e+07
1,Confections,6.633780e+07
2,Meat/Poultry,6.488115e+07
3,Dairy Products,5.801812e+07
4,Condiments,5.579513e+07


In [62]:
# Exercice 3 – Clients ayant dépensé plus que la moyenne
# --------------------------------------------------
# Contexte métier :
# Le service financier souhaite identifier les clients qui ont dépensé plus que la moyenne de tous les clients.
# Ce que tu dois retourner :
    # Le nom de la société
    # Le total dépensé par ce client (SUM(Quantity * UnitPrice * (1 - Discount)))
# Contraintes :
    # Utiliser une sous-requête dans WHERE pour comparer le total du client à la moyenne générale
# Tables concernées : Customers, Orders, "Order Details"

query = '''
SELECT
    customers.companyname,
    SUM("order details".Quantity * "order details".UnitPrice * (1 - "order details".Discount)) AS totaldepense
FROM
    customers
JOIN
    orders
    ON customers.customerid = orders.customerid
JOIN
    "order details"
    ON orders.orderid = "order details".orderid
GROUP BY
    customers.companyname
HAVING
    totaldepense >
    (
    SELECT
        AVG(nb)
    FROM
        (
        SELECT
            SUM("order details".Quantity * "order details".UnitPrice * (1 - "order details".Discount)) AS nb
        FROM
            customers
        JOIN
            orders
            ON customers.customerid = orders.customerid
        JOIN
            "order details"
            ON orders.orderid = "order details".orderid
        GROUP BY
            customers.companyname
        ) AS moyennenbcommandes
    )      
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Peut-on utiliser une fonction d’agrégation dans une sous-requête ?
# Réponse : Oui, à condition qu’elle retourne une seule valeur si on l’utilise dans SELECT, WHERE ou HAVING.

# Questions entretien : Dans quel cas est-il préférable d'utiliser une sous-requête dans WHERE plutôt qu’un JOIN ?
# Réponse : Quand on a besoin de comparer un résultat à une valeur agrégée ou à un ensemble calculé indépendamment, sans vouloir exposer toutes les colonnes dans le résultat final.

,CompanyName,totaldepense
0,Ana Trujillo Emparedados y helados,5534356.65
1,B's Beverages,6154115.34
2,Bottom-Dollar Markets,5064202.34
3,Bólido Comidas preparadas,5398064.44
4,Chop-suey Chinese,4989158.99


In [63]:
# Exercice 4 – Produits les plus chers par fournisseur (sous-requête corrélée)
# --------------------------------------------------
# Contexte métier :
# Le service achat veut lister, pour chaque fournisseur, le(s) produit(s) le(s) plus cher(s) de son catalogue.
# Ce que tu dois retourner :
    # Le nom du fournisseur (CompanyName)
    # Le nom du produit (ProductName)
    # Le prix unitaire (UnitPrice)
# Contraintes :
    # Utiliser une SOUS-REQUÊTE CORRÉLÉE pour comparer le prix du produit au prix MAX du même fournisseur.
# Tables concernées : Suppliers, Products

query = '''
SELECT
    suppliers.companyname,
    p.productname,
    p.unitprice
FROM
    suppliers
JOIN
    products p
    ON suppliers.supplierid = p.supplierid
WHERE
    p.UnitPrice = 
        (
        SELECT MAX(p2.UnitPrice)
        FROM Products p2
        WHERE p2.SupplierID = p.SupplierID
        )
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Qu’est-ce qu’une sous-requête corrélée et en quoi diffère-t-elle d’une sous-requête classique ?
# Réponse : Une sous-requête corrélée référence des colonnes de la requête externe et est ré-évaluée pour chaque ligne (ici p2.SupplierID = p.SupplierID).
#           Une sous-requête non corrélée est indépendante, s’exécute une fois, et son résultat est réutilisé.

# Questions entretien : Pourquoi peut-elle être plus lente sur de gros volumes et comment l’optimiser ?
# Réponse : Plus lente car évaluée ligne par ligne (effet N requêtes).

,CompanyName,ProductName,UnitPrice
0,Exotic Liquids,Chang,19.0
1,New Orleans Cajun Delights,Chef Anton's Cajun Seasoning,22.0
2,Grandma Kelly's Homestead,Northwoods Cranberry Sauce,40.0
3,Tokyo Traders,Mishi Kobe Niku,97.0
4,Cooperativa de Quesos 'Las Cabras',Queso Manchego La Pastora,38.0


In [64]:
# Exercice 5 – [Requête cassée] Produits au-dessus de la moyenne de leur catégorie
# --------------------------------------------------
# Contexte métier :
# Le marketing veut la liste des produits dont le prix est supérieur à la moyenne de leur catégorie.
# Ce que tu dois retourner :
    # Le nom du produit (ProductName)
    # Le prix unitaire (UnitPrice)
    # Le nom de la catégorie (CategoryName)
# Contraintes :
    # Corriger la sous-requête (pas de CTE / WITH)
    # La comparaison doit se faire à la moyenne de la même catégorie que le produit
# Tables concernées : Products, Categories

query = '''
SELECT 
    p.ProductName,
    p.UnitPrice,
    c.CategoryName
FROM Products p
JOIN Categories c ON c.CategoryID = p.CategoryID
WHERE p.UnitPrice > (
    SELECT AVG(p2.UnitPrice)
    FROM Products p2
    WHERE p2.CategoryID = p.CategoryID
)
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Pourquoi cette sous-requête pose problème telle quelle ?
# Réponse : Parce qu’elle renvoyait plusieurs lignes (une moyenne par catégorie) sans corrélation avec le produit courant. Une comparaison > exige une valeur scalaire (une seule ligne).

# Questions entretien : Comment transformer cette sous-requête pour qu’elle ne retourne qu’une seule valeur par ligne comparée ?
# Réponse : En la rendant corrélée au produit courant.

,ProductName,UnitPrice,CategoryName
0,Grandma's Boysenberry Spread,25.0,Condiments
1,Northwoods Cranberry Sauce,40.0,Condiments
2,Mishi Kobe Niku,97.0,Meat/Poultry
3,Ikura,31.0,Seafood
4,Queso Manchego La Pastora,38.0,Dairy Products


In [65]:
# Exercice 6 – Clients ayant commandé un produit contenant un mot-clé
# --------------------------------------------------
# Contexte métier :
# Le marketing veut cibler les clients qui ont déjà commandé un produit contenant un mot-clé (ex : "choco").
# Ce que tu dois retourner :
    # Le nom de la société (CompanyName)
    # Le nombre de commandes passées contenant au moins un produit correspondant au mot-clé
# Contraintes :
    # Utiliser une SOUS-REQUÊTE dans WHERE (pas de WITH)
    # Filtrer les produits par un mot-clé via LIKE (ex : '%choco%')
# Tables concernées : Customers, Orders, "Order Details", Products

query = '''
SELECT
    c.CompanyName,
    COUNT(DISTINCT o.OrderID) AS nbCommandes
FROM Customers c
JOIN Orders o ON o.CustomerID = c.CustomerID
WHERE o.OrderID IN (
    SELECT od.OrderID
    FROM "Order Details" od
    JOIN Products p ON p.ProductID = od.ProductID
    WHERE p.ProductName LIKE '%choco%'
)
GROUP BY c.CompanyName
ORDER BY nbCommandes DESC;
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle est la différence entre LIKE et INSTR() pour chercher un motif ?
# Réponse : LIKE utilise des jokers (% et _) pour faire des correspondances de motifs, INSTR() renvoie la position d’un motif (0/NULL s’il n’existe pas) ; on teste alors INSTR(col,'motif') > 0.

# Questions entretien : Pourquoi préférer une sous-requête dans WHERE plutôt qu’un JOIN direct dans ce cas ?
# Réponse : Pour exprimer “existe au moins un produit correspondant dans la commande” sans dupliquer les lignes par produit ; la sous-requête sur OrderID évite les doublons et garde la requête principale simple.


,CompanyName,nbCommandes
0,IT,224
1,Hungry Coyote Import Store,140
2,Princesa Isabel Vinhos,139
3,Ricardo Adocicados,136
4,B's Beverages,134


In [66]:
# Exercice 7 – Normalisation des villes (version simple)
# --------------------------------------------------
# Objectif : trouver les villes qui ont plusieurs écritures différentes.
# Étapes :
# 1) Normaliser très simplement (UPPER + TRIM)
# 2) Regrouper par ville normalisée
# 3) Garder les groupes avec au moins 2 variantes

query = '''
SELECT 
    NormalizedCity,
    COUNT(DISTINCT OriginalCity) AS NbVariantes,
    GROUP_CONCAT(DISTINCT OriginalCity) AS Variantes
FROM (
    SELECT
        c.City AS OriginalCity,
        UPPER(TRIM(c.City)) AS NormalizedCity
    FROM Customers c
)
GROUP BY NormalizedCity
HAVING COUNT(DISTINCT OriginalCity) >= 2
ORDER BY NbVariantes DESC;
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Pourquoi faire la normalisation dans une sous-requête FROM ici ?
# Réponse : Car on analyse une table entière

# Questions entretien : Quelles limites a cette normalisation minimale (UPPER/TRIM) ?
# Réponse : Toutes les autres variantes (accents, tirets, points...)

,NormalizedCity,NbVariantes,Variantes


In [67]:
# Exercice 8 – Produits avec noms trop courts ou trop longs (LENGTH + sous-requête)
# --------------------------------------------------
# Contexte métier :
# Le catalogue veut repérer les produits dont le nom est atypique (très court ou très long).
# Ce que tu dois retourner :
    # Le nom du produit (ProductName)
    # La longueur du nom (NameLen)
    # Un flag 'COURT' (< 5), 'LONG' (> 30), sinon 'OK'
# Contraintes :
    # Calculer LENGTH(ProductName) et le flag dans une SOUS-REQUÊTE placée dans FROM (pas de WITH)
    # Puis filtrer dans la requête externe pour garder seulement COURT ou LONG
# Tables concernées : Products

query = '''
SELECT
    *
FROM
    (
    SELECT
        products.productname,
        LENGTH(products.productname) AS TailleNom,
        CASE
            WHEN LENGTH(products.productname) > 30 THEN 'LONG'
            WHEN LENGTH(products.productname) < 5 THEN 'COURT'
            ELSE 'OK'
            END AS AnalyseTailleNom
    FROM
        products
    ) AS AnalyseNoms
WHERE 
    analysetaillenom NOT LIKE 'OK'
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : LENGTH() retourne-t-il une longueur en caractères ou en octets ? (et pourquoi ça peut compter)
# Réponse : En caractère, les octets sont une autre forme de mesure.

# Questions entretien : Pourquoi calculer LENGTH + CASE dans une sous-requête FROM plutôt que directement dans le SELECT final ?
# Réponse : Parce qu'on aime se compliquer la vie ^^

,productname,TailleNom,AnalyseTailleNom
0,Chai,4,COURT
1,Uncle Bob's Organic Dried Pears,31,LONG
2,Tofu,4,COURT
3,Jack's New England Clam Chowder,31,LONG
4,Louisiana Fiery Hot Pepper Sauce,32,LONG


In [68]:
# Exercice 9 – Commandes passées le même mois/année qu’une commande donnée
# --------------------------------------------------
# Contexte métier :
# Le service analyse veut identifier toutes les commandes passées le même mois et la même année qu’une commande spécifique (ex : OrderID = 10250).
# Ce que tu dois retourner :
    # L’ID de la commande
    # La date de la commande
    # L’ID du client
# Contraintes :
    # Utiliser une SOUS-REQUÊTE pour extraire le mois et l’année de la commande de référence
    # Pas de CTE / WITH
# Tables concernées : Orders

query = '''
SELECT
    orders.orderid,
    orders.orderdate,
    orders.customerid
FROM
    orders
WHERE
    strftime('%Y', orders.orderdate) = 
        (
        SELECT
            strftime('%Y', orders.orderdate)
        FROM
            orders
        WHERE
            orderid = 10250
        )
    AND
    strftime('%m', orders.orderdate) = 
        (
        SELECT
            strftime('%m', orders.orderdate)
        FROM
            orders
        WHERE
            orderid = 10250
        )    
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Comment extraire le mois ou l’année d’une date en SQL ?
# Réponse : Ca dépend des types de SQL.

# Questions entretien : Qu’est-ce qu’une sous-requête corrélée et pourrait-on en utiliser une ici ?
# Réponse : 

,OrderID,OrderDate,CustomerID
0,10248,2016-07-04,VINET
1,10249,2016-07-05,TOMSP
2,10250,2016-07-08,HANAR
3,10251,2016-07-08,VICTE
4,10252,2016-07-09,SUPRD


In [69]:
# Exercice 10 – Produits commandés uniquement par un seul client
# --------------------------------------------------
# Contexte métier :
# Le service commercial veut repérer les produits qui n'ont été achetés que par un seul client.
# Ce que tu dois retourner :
    # Le nom du produit (ProductName)
    # Le nom du client (CompanyName)
    # Le nombre total de commandes de ce produit par ce client
# Contraintes :
    # Utiliser une SOUS-REQUÊTE dans WHERE pour isoler les produits achetés par un seul client
# Tables concernées : Products, Orders, "Order Details", Customers

query = '''
SELECT
    p.productname,
    c.companyname,
    COUNT(o.orderid) AS nbCommande
FROM
    products p
JOIN
    "order details" od ON p.productid = od.productid
JOIN
    orders o ON od.orderid = o.orderid
JOIN
    customers c ON o.customerid = c.customerid
WHERE
    p.productid IN (
        SELECT
            od2.productid
        FROM
            "order details" od2
        JOIN
            orders o2 ON od2.orderid = o2.orderid
        GROUP BY
            od2.productid
        HAVING
            COUNT(DISTINCT o2.customerid) = 1
    )
GROUP BY
    p.productname,
    c.companyname
ORDER BY
    p.productname;
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Comment identifier les valeurs uniques dans un jeu de données via SQL ?
# Réponse : On peut utiliser GROUP BY avec HAVING COUNT(DISTINCT colonne) = 1 pour trouver les valeurs présentes une seule fois.

# Questions entretien : Avantages et limites d'une sous-requête par rapport à un GROUP BY direct ?
# Réponse : La sous-requête rend la logique plus claire quand on veut isoler d’abord un ensemble précis, mais elle peut être moins performante que de tout faire en une seule requête avec un GROUP BY bien pensé.

,ProductName,CompanyName,nbCommande


## Exercices du programme semaine 3 : 12 Aout 2025

In [70]:
# Exercice 1 – Évolution du montant des commandes par client
# --------------------------------------------------
# Contexte métier :
# Le service commercial veut analyser l'évolution des montants de commandes pour chaque client.
# Ce que tu dois retourner :
    # Le nom du client (CompanyName)
    # L'ID de la commande (OrderID)
    # Le montant total de la commande
    # Le montant de la commande précédente du même client
    # La différence entre les deux
# Contraintes :
    # Utiliser LAG() pour obtenir le montant précédent
# Tables concernées : Customers, Orders, "Order Details"

query = '''
WITH commandes AS (
    SELECT
        customers.companyname,
        orders.orderid,
        orders.orderdate,
        SUM(("order details".unitprice - "order details".discount) * "order details".quantity) AS MontantTotalCommande
    FROM
        customers
    JOIN orders ON customers.customerid = orders.customerid
    JOIN "order details" ON orders.orderid = "order details".orderid
    GROUP BY
        orders.orderid
)
SELECT
    companyname,
    orderid,
    orderdate,
    MontantTotalCommande,
    LAG(MontantTotalCommande) OVER(PARTITION BY companyname ORDER BY orderdate) AS MontantTotalCommandePrecedente,
    MontantTotalCommande - LAG(MontantTotalCommande) OVER(PARTITION BY companyname ORDER BY orderdate) AS Difference
FROM commandes
ORDER BY
    companyname, orderdate
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : À quoi sert la fonction LAG() ?
# Réponse : A montrer la valeur précédente selon un champ défini.

# Questions entretien : Différence entre LAG() et une auto-jointure pour récupérer la ligne précédente ?
# Réponse : Plus rapide, plus efficace et plus lisible.

,companyname,orderid,orderdate,MontantTotalCommande,MontantTotalCommandePrecedente,Difference
0,Alfreds Futterkiste,12671,2012-08-22 13:27:12,6959.05,NaN,NaN
1,Alfreds Futterkiste,12078,2012-08-31 17:02:02,19811.50,6959.05,12852.45
2,Alfreds Futterkiste,11908,2012-09-07 02:39:28,37222.19,19811.50,17410.69
3,Alfreds Futterkiste,14208,2012-10-06 12:09:23,35474.20,37222.19,-1747.99
4,Alfreds Futterkiste,19569,2012-12-16 21:52:46,18159.50,35474.20,-17314.70


In [71]:
# Exercice 2 – Classement des produits par montant total vendu
# --------------------------------------------------
# Contexte métier :
# Le service marketing veut établir un classement des produits en fonction du montant total vendu, en montrant également le classement dans leur propre catégorie.
# Ce que tu dois retourner :
    # Le nom de la catégorie (CategoryName)
    # Le nom du produit (ProductName)
    # Le montant total vendu (TotalVentes)
    # Le rang du produit dans sa catégorie (par montant décroissant)
# Contraintes :
    # Utiliser RANK() avec PARTITION BY catégorie
# Tables concernées : Categories, Products, "Order Details"

query = '''
SELECT
    categories.categoryname,
    products.productname,
    SUM("order details".unitprice * (1 - "order details".discount) * "order details".quantity) AS MontantTotalVente,
    RANK() OVER(
        PARTITION BY categories.categoryname 
        ORDER BY SUM("order details".unitprice * (1 - "order details".discount) * "order details".quantity) DESC
    ) AS Rank
FROM
    categories
JOIN
    products ON categories.categoryid = products.categoryid
JOIN
    "order details" ON products.productid = "order details".productid
GROUP BY
    categories.categoryname, products.productname
ORDER BY
    categories.categoryname, Rank
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle est la différence entre RANK() et DENSE_RANK() ?
# Réponse : RANK() laisse des "trous" dans le classement si deux lignes ont la même valeur, alors que DENSE_RANK() ne laisse pas de trou.

# Questions entretien : Que fait PARTITION BY dans une fonction de fenêtre ?
# Réponse : Il divise les données en groupes et applique la fonction de fenêtre séparément à chaque groupe.

,CategoryName,ProductName,MontantTotalVente,Rank
0,Beverages,Côte de Blaye,5.326590e+07,1
1,Beverages,Ipoh Coffee,9.333375e+06,2
2,Beverages,Chang,3.830511e+06,3
3,Beverages,Steeleye Stout,3.640338e+06,4
4,Beverages,Chartreuse verte,3.634327e+06,5


In [72]:
# Exercice 3 – Évolution des ventes mensuelles par catégorie
# --------------------------------------------------
# Contexte métier :
# Le service marketing souhaite suivre l'évolution des ventes mensuelles pour chaque catégorie de produits.
# Ce que tu dois retourner :
    # Le mois (au format YYYY-MM)
    # Le nom de la catégorie (CategoryName)
    # Le montant total des ventes pour ce mois et cette catégorie
    # La différence par rapport au mois précédent (même catégorie)
# Contraintes :
    # Utiliser LAG() pour obtenir le montant du mois précédent
# Tables concernées : Categories, Products, "Order Details", Orders

query = '''
WITH tablemoiscommande AS 
    (
    SELECT
        categories.categoryname,
        strftime('%Y %m', orders.orderdate) AS MoisCommande,
        SUM("order details".unitprice * (1 - "order details".discount) * "order details".quantity) AS MontantTotalVente
    FROM
        categories
    JOIN
        products ON categories.categoryid = products.categoryid
    JOIN
        "order details" ON products.productid = "order details".productid
    JOIN
        orders ON "order details".orderid = orders.orderid
    GROUP BY
        MoisCommande, categories.categoryname
    ORDER BY
        categories.categoryname, MoisCommande DESC
    )

SELECT
    categoryname,
    MoisCommande,
    MontantTotalVente,
    LAG(MontantTotalVente) OVER(PARTITION BY categoryname ORDER BY MoisCommande) AS MontantMoisPrecedent
FROM 
    tablemoiscommande
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle différence entre LAG() et LEAD() ?
# Réponse : LEAD() prends la valeur suivante.

# Questions entretien : Pourquoi doit-on inclure ORDER BY dans une fonction de fenêtre comme LAG() ?
# Réponse : Pour lui indiquer l'order qu'il doit avoir pour prendre la valeur précédente. 

,categoryname,MoisCommande,MontantTotalVente,MontantMoisPrecedent
0,Beverages,2012 07,422541.00,NaN
1,Beverages,2012 08,717189.75,422541.00
2,Beverages,2012 09,691794.00,717189.75
3,Beverages,2012 10,657336.50,691794.00
4,Beverages,2012 11,622583.00,657336.50


In [73]:
# Exercice 4 – TOP N commandes par montant pour chaque client
# --------------------------------------------------
# Contexte métier :
# Le service commercial veut identifier les commandes les plus importantes de chaque client, par montant total.
# Ce que tu dois retourner :
    # Le nom du client (CompanyName)
    # L'ID de la commande (OrderID)
    # Le montant total de la commande (TotalCommande)
    # Le rang de la commande pour ce client (par montant décroissant)
# Contraintes :
    # Utiliser ROW_NUMBER() avec PARTITION BY client
    # Ne garder que les 3 premières commandes par client
# Tables concernées : Customers, Orders, "Order Details"

query = '''
WITH meilleurscommandes AS
    (
    SELECT
        customers.companyname,
        orders.orderid,
        SUM("order details".unitprice * (1 - "order details".discount) * "order details".quantity) AS MontantTotalCommande,
        ROW_NUMBER() OVER(PARTITION BY customers.companyname ORDER BY SUM("order details".unitprice * (1 - "order details".discount) * "order details".quantity) DESC) AS Rank
    FROM
        customers
    JOIN 
        orders ON customers.customerid = orders.customerid
    JOIN
        "order details" ON "order details".orderid = orders.orderid
    GROUP BY
        customers.companyname, orders.orderid
    )

SELECT
    companyname,
    orderid,
    MontantTotalCommande,
    Rank
FROM 
    meilleurscommandes
WHERE 
    Rank <= 3
ORDER BY 
    companyname, Rank
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle est la différence entre ROW_NUMBER(), RANK() et DENSE_RANK() ?
# Réponse : ROW_NUMBER() : numérote 1,2,3,… sans tenir compte des ex æquo (les égalités auront des numéros différents). RANK() : ex æquo partagent le même rang et on saute les rangs suivants (1,1,3,…). DENSE_RANK() : ex æquo partagent le même rang sans saut (1,1,2,…).

# Questions entretien : Pourquoi utiliser PARTITION BY ici ?
# Réponse : Pour avoir un odre par companyname.

,companyname,orderid,MontantTotalCommande,Rank
0,Alfreds Futterkiste,17684,61769.41,1
1,Alfreds Futterkiste,18421,59934.52,2
2,Alfreds Futterkiste,14252,58015.87,3
3,Ana Trujillo Emparedados y helados,16433,64990.14,1
4,Ana Trujillo Emparedados y helados,12033,62992.66,2


In [74]:
# Exercice 5 – Montant cumulé des ventes par client
# --------------------------------------------------
# Contexte métier :
# Le service commercial veut suivre, pour chaque client, l’évolution du cumul de ses ventes au fil des commandes.
# Ce que tu dois retourner :
    # Le nom du client (CompanyName)
    # L’ID de la commande (OrderID)
    # Le montant de la commande (TotalCommande)
    # Le montant cumulé des ventes du client à date (CumulClient)
# Contraintes :
    # Utiliser SUM() OVER (PARTITION BY client ORDER BY date de commande)
    # Soigner l’ordre (ORDER BY) pour que le cumul soit chronologique
# Tables concernées : Customers, Orders, "Order Details"

query = '''
WITH CompanyTotalCommande AS
    (
    SELECT
        customers.companyname,
        orders.orderid,
        SUM("order details".unitprice * (1 - "order details".discount) * "order details".quantity) AS MontantTotalCommande
    FROM
        customers
    JOIN
        orders ON customers.customerid = orders.customerid
    JOIN
        "order details" ON orders.orderid = "order details".orderid
    GROUP BY
        customers.companyname, orders.orderid
    )

SELECT
    companyname,
    orderid,
    montanttotalcommande,
    SUM(MontantTotalCommande) OVER(PARTITION BY companyname ORDER BY orderid) AS CumulClient
FROM
    CompanyTotalCommande
ORDER BY
    companyname, orderid DESC
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle différence entre SUM() avec GROUP BY et SUM() OVER(...) ?
# Réponse : SUM() aevc GROUP BY fait la somme des ventes pour le group spécifié. SUM() OVER() Fait une domme cummulé en fonction d'un ordre et d'une partition.

# Questions entretien : Pourquoi l’ORDER BY dans la fenêtre est-il essentiel pour un cumul ?
# Réponse : Pour lui spécifier dans quel ordre on veut qu'il cumule ici l'orderid (qui équivaut à orderdate au final)

,companyname,orderid,MontantTotalCommande,CumulClient
0,Alfreds Futterkiste,26469,42707.97,3965464.95
1,Alfreds Futterkiste,26376,19398.50,3922756.98
2,Alfreds Futterkiste,26255,53452.08,3903358.48
3,Alfreds Futterkiste,26123,1772.00,3849906.40
4,Alfreds Futterkiste,25970,35183.24,3848134.40


In [75]:
# Exercice 6 – Comparaison au panier moyen du client (AVG() OVER)
# --------------------------------------------------
# Contexte métier :
# Le service commercial veut savoir, pour chaque commande, si son montant est supérieur ou inférieur au panier moyen habituel du client.
# Ce que tu dois retourner :
    # Le nom du client (CompanyName)
    # L’ID de la commande (OrderID)
    # Le montant de la commande (TotalCommande)
    # Le panier moyen du client (MoyenneClient) via AVG() OVER(PARTITION BY client)
    # Un indicateur 'AuDessusMoyenne' = 'OUI' si TotalCommande > MoyenneClient, sinon 'NON'
# Contraintes :
    # Calculer le montant de commande (UnitPrice * (1 - Discount) * Quantity)
    # Utiliser une fenêtre AVG() OVER (PARTITION BY client)
# Tables concernées : Customers, Orders, "Order Details"

query = '''
WITH CompanyTotalCommande AS (
  SELECT
    c.CompanyName,
    o.OrderID,
    SUM(od.UnitPrice * (1 - od.Discount) * od.Quantity) AS MontantTotalCommande
  FROM Customers c
  JOIN Orders o ON c.CustomerID = o.CustomerID
  JOIN "order details" od ON od.OrderID = o.OrderID
  GROUP BY c.CompanyName, o.OrderID
),
AvecMoyenne AS (
  SELECT
    CompanyName,
    OrderID,
    MontantTotalCommande,
    AVG(MontantTotalCommande) OVER (PARTITION BY CompanyName) AS MoyenneClient
  FROM CompanyTotalCommande
)
SELECT
  CompanyName,
  OrderID,
  MontantTotalCommande,
  ROUND(MoyenneClient, 2) AS MoyenneClient,
  CASE 
      WHEN MontantTotalCommande > MoyenneClient THEN 'OUI' 
      ELSE 'NON' 
      END AS AuDessusMoyenne
FROM 
    AvecMoyenne
ORDER BY 
    CompanyName, OrderID DESC;
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle différence entre AVG() avec GROUP BY et AVG() OVER(PARTITION BY ...) ?
# Réponse : Même principe pour l'exo précédent avec le SUM() et SUM() OVER()

# Questions entretien : Dans quel ordre exécuter : agrégation par commande vs fonction de fenêtre ?
# Réponse : On agrége en premier puis on fait la fonction de fenêtre. 

,CompanyName,OrderID,MontantTotalCommande,MoyenneClient,AuDessusMoyenne
0,Alfreds Futterkiste,26469,42707.97,24328.01,OUI
1,Alfreds Futterkiste,26376,19398.50,24328.01,NON
2,Alfreds Futterkiste,26255,53452.08,24328.01,OUI
3,Alfreds Futterkiste,26123,1772.00,24328.01,NON
4,Alfreds Futterkiste,25970,35183.24,24328.01,OUI


In [76]:
# Exercice 7 – Produit le plus vendu chaque mois
# --------------------------------------------------
# Contexte métier :
# La direction produit veut identifier, pour chaque mois, le(s) produit(s) le(s) plus vendu(s) en montant.
# Ce que tu dois retourner :
    # Le mois (YYYY-MM)
    # Le nom du produit (ProductName)
    # Le montant total vendu sur le mois (TotalMois)
    # Un indicateur TopDuMois = 'OUI' si le produit est n°1 du mois (ex aequo inclus), sinon 'NON'
# Contraintes :
    # Calculer le total mensuel par produit
    # Utiliser RANK() OVER(PARTITION BY Mois ORDER BY TotalMois DESC)
# Tables concernées : Products, "Order Details", Orders

query = '''
WITH TotalMois AS
    (
    SELECT
        products.productname,
        strftime('%Y-%m', orders.orderdate) AS MoisCommande,
        SUM("order details".unitprice * (1 - "order details".discount) * "order details".quantity) AS MontantTotalMois
    FROM
        products
    JOIN
        "order details" ON products.productid = "order details".productid
    JOIN
        orders ON "order details".orderid = orders.orderid
    GROUP BY
        products.productname, MoisCommande
    ),
TotalMoisRanking AS
    (
    SELECT
        productname,
        moiscommande,
        montanttotalmois,
        RANK() OVER(PARTITION BY moiscommande ORDER BY montanttotalmois DESC) AS Rank
    FROM
        TotalMois
    )

SELECT
    productname,
    moiscommande,
    montanttotalmois,
    rank,
    CASE
        WHEN rank = 1 THEN 'OUI'
        ELSE 'NON'
        END AS TopDuMois
FROM
    totalmoisranking
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle différence entre RANK() et DENSE_RANK() pour ce cas mensuel ?
# Réponse : DENSE_RANK continue la numérotation meme si il y a des égalités. RANK() lui va sauter des nombres.

# Questions entretien : Pourquoi partitionner par Mois dans la fenêtre ?
# Réponse : Car on veut voir le top 1 par mois et non le top mois par produit.

,productname,moiscommande,montanttotalmois,Rank,TopDuMois
0,Côte de Blaye,2012-07,251906.00,1,OUI
1,Mishi Kobe Niku,2012-07,90016.00,2,NON
2,Thüringer Rostbratwurst,2012-07,87024.37,3,NON
3,Sir Rodney's Marmalade,2012-07,68445.00,4,NON
4,Carnarvon Tigers,2012-07,56875.00,5,NON


In [77]:
# Exercice 8 – Part de marché d’un produit dans sa catégorie
# --------------------------------------------------
# Contexte métier :
# Le marketing veut connaître, pour chaque produit, sa part de marché (en montant) au sein de sa catégorie.
# Ce que tu dois retourner :
    # Le nom de la catégorie (CategoryName)
    # Le nom du produit (ProductName)
    # Le montant total vendu du produit (TotalProduit)
    # Le montant total de la catégorie (TotalCategorie)
    # La part de marché du produit dans sa catégorie (PartDeMarche = TotalProduit / TotalCategorie)
# Contraintes :
    # Calculer TotalProduit avec SUM(Quantity * UnitPrice * (1 - Discount))
    # Utiliser SUM() OVER (PARTITION BY catégorie) pour TotalCategorie (fenêtre)
    # Afficher la part en décimal ou pourcentage (au choix)
# Tables concernées : Categories, Products, "Order Details"

query = '''
WITH totalventeproducts AS
    (
    SELECT
        categories.categoryname,
        products.productname,
        SUM("order details".unitprice * (1 - "order details".discount) * "order details".quantity) AS MontantTotalproduct
    FROM
        categories
    JOIN
        products ON categories.categoryid = products.categoryid
    JOIN
        "order details" ON products.productid = "order details".productid
    GROUP BY
        categories.categoryname, products.productname
    ),
totalventecategories AS
    (
    SELECT
        categoryname,
        productname,
        montanttotalproduct,
        SUM(montanttotalproduct) OVER(PARTITION BY categoryname) AS montanttotalcategorie
    FROM
        totalventeproducts
    )

SELECT
    categoryname,
    productname,
    montanttotalproduct,
    montanttotalcategorie,
    ROUND((montanttotalproduct * 100 / montanttotalcategorie),2) AS PourcentageCategorie
FROM
    totalventecategories
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle différence entre SUM() avec GROUP BY et SUM() OVER(PARTITION BY ...) dans ce cas ?
# Réponse : Le SUM avec GROUP BY() fais la somme de ce qui est groupé, avec le PARTITION BY on fais la somme de ce qui est partitionné. 

# Questions entretien : Pourquoi PARTITION BY (catégorie) est indispensable ici ?
# Réponse : Pour faire la somme uniquement sur chaque catégorie et non la somme globale. 

,categoryname,productname,montanttotalproduct,montanttotalcategorie,PourcentageCategorie
0,Beverages,Chai,3.632174e+06,92163184.18,3.94
1,Beverages,Chang,3.830511e+06,92163184.18,4.16
2,Beverages,Chartreuse verte,3.634327e+06,92163184.18,3.94
3,Beverages,Côte de Blaye,5.326590e+07,92163184.18,57.80
4,Beverages,Guaraná Fantástica,9.082574e+05,92163184.18,0.99


In [78]:
# Exercice 9 – Écart au montant moyen global (AVG() OVER sans PARTITION)
# --------------------------------------------------
# Contexte métier :
# Le management veut savoir quels produits performent au-dessus ou en dessous de la moyenne globale (tous produits confondus).
# Ce que tu dois retourner :
    # Le nom du produit (ProductName)
    # Le montant total vendu du produit (TotalProduit)
    # La moyenne globale des montants produits (MoyenneGlobale) via AVG() OVER() SANS PARTITION
    # L’écart absolu (TotalProduit - MoyenneGlobale)
    # L’écart en % (TotalProduit / MoyenneGlobale - 1)
# Contraintes :
    # Calculer d’abord le TotalProduit par produit
    # Puis appliquer AVG() OVER() (sans PARTITION) sur ces totaux
# Tables concernées : Products, "Order Details"

query = '''
WITH totalproduct AS
    (
    SELECT
        products.productname,
        SUM("order details".unitprice * (1 - "order details".discount) * "order details".quantity) AS montanttotalproduct
    FROM
        products
    JOIN
        "order details" ON products.productid = "order details".productid
    GROUP BY 
        products.productname
    ),
totalmoyenneproduct AS
    (
    SELECT
        productname,
        montanttotalproduct,
        AVG(montanttotalproduct) OVER() AS moyenneproduct
    FROM
        totalproduct
    )

SELECT
    productname,
    montanttotalproduct,
    moyenneproduct,
    montanttotalproduct - moyenneproduct AS differencevsmoyenne,
    (montanttotalproduct / moyenneproduct -1) AS pourcentageecart
FROM
    totalmoyenneproduct
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle est la différence entre AVG() avec GROUP BY et AVG() OVER() (sans PARTITION) ?
# Réponse : AVG() dans le groupe by fais la moyenne des produits en fonction de ce qui est groupé, le AVG() OVER() fait la moyenne de tout ici car sans partition et permet d'avoir le même résultat sur toutes les lignes sans les aggréger. 

# Questions entretien : Dans quel cas préférer AVG() OVER() (sans PARTITION) pour comparer chaque ligne au global ?
# Réponse : Quand on veut une colonne avec le même chiffre et non pas juste une ligne.

,productname,montanttotalproduct,moyenneproduct,differencevsmoyenne,pourcentageecart
0,Alice Mutton,7.884412e+06,5.823203e+06,2.061209e+06,0.353965
1,Aniseed Syrup,2.021624e+06,5.823203e+06,-3.801579e+06,-0.652833
2,Boston Crab Meat,3.681884e+06,5.823203e+06,-2.141319e+06,-0.367722
3,Camembert Pierrot,6.900443e+06,5.823203e+06,1.077240e+06,0.184991
4,Carnarvon Tigers,1.260467e+07,5.823203e+06,6.781469e+06,1.164560


In [79]:
# Exercice 10 – Classement global et par catégorie des produits
# --------------------------------------------------
# Contexte métier :
# Le service marketing veut établir un double classement des produits :
    # un classement global (tous produits confondus)
    # un classement à l’intérieur de chaque catégorie
# Ce que tu dois retourner :
    # Le nom de la catégorie (CategoryName)
    # Le nom du produit (ProductName)
    # Le montant total vendu du produit (TotalProduit)
    # Le rang global (RankGlobal)
    # Le rang dans la catégorie (RankCategorie)
# Contraintes :
    # Utiliser RANK() OVER() pour le classement global
    # Utiliser RANK() OVER(PARTITION BY catégorie) pour le classement par catégorie
# Tables concernées : Categories, Products, "Order Details"

query = '''
WITH TotalProduct AS (
  SELECT
    categories.CategoryName,
    products.ProductName,
    SUM("order details".UnitPrice * (1 - "order details".Discount) * "order details".Quantity) AS MontantTotalProduit
  FROM categories
  JOIN products ON categories.CategoryID = products.CategoryID
  JOIN "order details" ON products.ProductID = "order details".ProductID
  GROUP BY categories.CategoryName, products.ProductName
)
SELECT
  CategoryName,
  ProductName,
  MontantTotalProduit,
  RANK() OVER (ORDER BY MontantTotalProduit DESC) AS RankingGlobal,
  RANK() OVER (PARTITION BY CategoryName ORDER BY MontantTotalProduit DESC) AS RankingCategorie
FROM TotalProduct
ORDER BY CategoryName, RankingCategorie;
'''

df = requests_sql(query, conn)
df.head()

# Questions entretien : Quelle est la différence entre un RANK() OVER() et un RANK() OVER(PARTITION BY ...) ?
# Réponse : le PARTITION BY() permet de faire un ranking ici par catégorie alors que sans il fait le ranking global. 

# Questions entretien : Pourquoi faire deux classements différents dans cette requête ?
# Réponse : Pour connaitre à la fois le ranking dans chaque catégorie et dans l'ensemble des produits. 

,CategoryName,ProductName,MontantTotalProduit,RankingGlobal,RankingCategorie
0,Beverages,Côte de Blaye,5.326590e+07,1,1
1,Beverages,Ipoh Coffee,9.333375e+06,9,2
2,Beverages,Chang,3.830511e+06,41,3
3,Beverages,Steeleye Stout,3.640338e+06,44,4
4,Beverages,Chartreuse verte,3.634327e+06,45,5
